In [1]:
# designed to be run after 03-clinical_variables_final. this notebook does some data cleaning/processing. run before -___ notebook.

## cleans many aspects of the raw clinical variables.
## collapses and formats all of the various categorical variables into discrete variables as well. 

In [157]:
import pandas as pd
import matplotlib.pyplot as plt
import os
from pathlib import Path
import seaborn as sns
import numpy as np
import glob

from joblib import Memory
memory = Memory(cachedir='/tmp', verbose=0)
#@memory.cache above any def fxn.

%matplotlib inline
plt.style.use('ggplot')

from notebook.services.config import ConfigManager
cm = ConfigManager()
cm.update('livereveal', {
        'width': 1024,
        'height': 768,
        'scroll': True,
})

%load_ext autotime

The autotime extension is already loaded. To reload it, use:
  %reload_ext autotime
time: 16 ms (started: 2023-07-26 18:36:35 +08:00)


C:\Users\csfhe\AppData\Local\Temp\ipykernel_10128\3239421198.py:10: DeprecationWarning: The 'cachedir' parameter has been deprecated in version 0.12 and will be removed in version 0.14.
You provided "cachedir='/tmp'", use "location='/tmp'" instead.
  memory = Memory(cachedir='/tmp', verbose=0)


In [158]:
#patients of interest from rotation_cohort_generation
from parameters import final_pt_df_v, date, repository_path

#patients of interest from rotation_cohort_generation
final_pt_df2 = final_pt_df_v
del(final_pt_df_v)

patients= list(final_pt_df2['subject_id'].unique())
hadm_id= list(final_pt_df2['hadm_id'].unique())
stay_id= list(final_pt_df2['stay_id'].unique())
stay_id= [int(x) for x in stay_id]


time: 0 ns (started: 2023-07-26 18:36:37 +08:00)


In [27]:
final_pt_df2['final_bin'].value_counts()

final_bin
C_neg/A_partial    14583
C_neg/A_full        7802
C_pos/A_partial     2551
C_pos/A_full        1535
Name: count, dtype: int64

time: 15 ms (started: 2023-07-26 17:42:45 +08:00)


# extracting clinical data for our patients
## IMPORTANT, USE THIS TO TUNE TIMEWINDOW OF EXTRACTION AND FOLDER TO SAVE IN

#NOTE ON MY DF NAMING CONVENTION:
origionally when I coded this workbook, it was for 72 hour timewindows, so every dataframe had _72 at the end. this was changed on 6/5/19 and was made more generalizable by finding name of each corresponding df in the df list and using this variable.

In [28]:
from parameters import lower_window, upper_window, folder, date, time_col, time_var, patient_df, save_boolean


time: 0 ns (started: 2023-07-26 17:42:50 +08:00)


### begin pipeline:

# changing my code structure to be a dictionary of dataframes

In [52]:
#folder to save files to:
save_path= str(repository_path)+'/data/cleaned/'
#folder=None


def save_df(df, df_name='default', save_path=save_path, add_subfolder=False):
    #uses the date and supplied df name and saves to the savepath specified above.
    if df_name == 'default':
        df_name= "%s"%(df)
    
    address=save_path+'%s/'%(folder)
    if not os.path.exists(address):
        print(address)
        os.makedirs(address)
    pd.DataFrame(df).to_csv(Path(address+'%s_%s_prepped.csv' %(date, df_name)))


time: 16 ms (started: 2023-07-26 18:00:34 +08:00)


In [53]:
save_path+'%s/'%(folder)

'c:\\Users\\csfhe\\Documents\\RA\\mimiciii-antibiotics-opensource\\notebooks/data/cleaned/24_hr_window/'

time: 0 ns (started: 2023-07-26 18:00:38 +08:00)


In [54]:
##folder with all clinical variable csv's
allFiles = glob.glob(str(repository_path)+ '/data/raw/%s/'%(folder) + "{}_*.csv".format(date))

time: 0 ns (started: 2023-07-26 18:00:39 +08:00)


In [34]:
allFiles

['c:\\Users\\csfhe\\Documents\\RA\\mimiciii-antibiotics-opensource\\notebooks/data/raw/24_hr_window\\07242023_bg_all_nosummary.csv',
 'c:\\Users\\csfhe\\Documents\\RA\\mimiciii-antibiotics-opensource\\notebooks/data/raw/24_hr_window\\07242023_cancer_elix.csv',
 'c:\\Users\\csfhe\\Documents\\RA\\mimiciii-antibiotics-opensource\\notebooks/data/raw/24_hr_window\\07242023_gcs.csv',
 'c:\\Users\\csfhe\\Documents\\RA\\mimiciii-antibiotics-opensource\\notebooks/data/raw/24_hr_window\\07242023_heightfirstday.csv',
 'c:\\Users\\csfhe\\Documents\\RA\\mimiciii-antibiotics-opensource\\notebooks/data/raw/24_hr_window\\07242023_labs_all_nosummary.csv',
 'c:\\Users\\csfhe\\Documents\\RA\\mimiciii-antibiotics-opensource\\notebooks/data/raw/24_hr_window\\07242023_pt_info.csv',
 'c:\\Users\\csfhe\\Documents\\RA\\mimiciii-antibiotics-opensource\\notebooks/data/raw/24_hr_window\\07242023_rrt_merged.csv',
 'c:\\Users\\csfhe\\Documents\\RA\\mimiciii-antibiotics-opensource\\notebooks/data/raw/24_hr_window\\0

time: 0 ns (started: 2023-07-26 17:43:23 +08:00)


In [205]:
#making a dictionary of all my dataframes for easier cycling through

df_list=[]
for element in allFiles:
    df_list.append(element.split('{}_'.format(date))[1].split('.csv')[0]) #making an list of all my dataframes in order they appear in file

dfs = {}
i=0
for name in df_list:
    dfs[name] = pd.read_csv(allFiles[i],  index_col=0)
    i+=1

C:\Users\csfhe\AppData\Local\Temp\ipykernel_10128\763664031.py:10: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  dfs[name] = pd.read_csv(allFiles[i],  index_col=0)


time: 9.25 s (started: 2023-07-26 19:07:51 +08:00)


In [160]:
df_list

['bg_all_nosummary',
 'cancer_elix',
 'gcs',
 'heightfirstday',
 'labs_all_nosummary',
 'pt_info',
 'rrt_merged',
 'sum_elix',
 'vaso_dose',
 'ventcategory',
 'vitals_all_nosummary',
 'weightfirstday']

time: 0 ns (started: 2023-07-26 18:36:57 +08:00)


In [58]:
#assigning the appropriate name to each df in a flexible way

indices = [i for i, s in enumerate(df_list) if 'bg' in s]
bg_df= df_list[indices[0]]

indices = [i for i, s in enumerate(df_list) if 'cancer' in s]
cancer_elix_df= df_list[indices[0]]

# indices = [i for i, s in enumerate(df_list) if 'uti' in s]
# uti_df= df_list[indices[0]]

indices = [i for i, s in enumerate(df_list) if 'vent' in s]
vent_df= df_list[indices[0]]

indices = [i for i, s in enumerate(df_list) if 'vitals' in s]
vitals_df= df_list[indices[0]]

indices = [i for i, s in enumerate(df_list) if 'vaso' in s]
vaso_df= df_list[indices[0]]

indices = [i for i, s in enumerate(df_list) if 'pt_info' in s]
pt_info_df= df_list[indices[0]]

indices = [i for i, s in enumerate(df_list) if 'gcs' in s]
gcs_df= df_list[indices[0]]

indices = [i for i, s in enumerate(df_list) if 'sum_elix' in s]
sum_elix_df = df_list[indices[0]]

# indices = [i for i, s in enumerate(df_list) if 'sofa' in s]
# sofa_df= df_list[indices[0]]

indices = [i for i, s in enumerate(df_list) if 'weight' in s]
weight_df= df_list[indices[0]]

indices = [i for i, s in enumerate(df_list) if 'labs' in s]
labs_df= df_list[indices[0]]

indices = [i for i, s in enumerate(df_list) if 'height' in s]
height_df= df_list[indices[0]]

indices = [i for i, s in enumerate(df_list) if 'rrt' in s]
rrt_df= df_list[indices[0]]

time: 15 ms (started: 2023-07-26 18:01:06 +08:00)


In [59]:
#adding a t_0 to each df that doesn't currently have it

for element in df_list:
    #print(element,':',list(dfs[element]))
    if ('t_0' in list(dfs[element]))==False and 'stay_id' in list(dfs[element]) :
        #print("true")
        dfs[element]= pd.merge(dfs[element], final_pt_df2[['stay_id','t_0']], how='left')
    elif ('t_0' in list(dfs[element]))==False and 'hadm_id' in list(dfs[element]) :
        #print("true")
        dfs[element]= pd.merge(dfs[element], final_pt_df2[['hadm_id','t_0']], how='left')
    else:
        print("false")


false
false
false
false
false
false
false
false
time: 63 ms (started: 2023-07-26 18:01:16 +08:00)


In [60]:
for element in df_list:
    print(element,':',list(dfs[element]))

bg_all_nosummary : ['subject_id', 'hadm_id', 'stay_id', 'charttime', 'label', 'valuenum', 'value', 'valueuom', 'unique_var', 't_0', 'delta']
cancer_elix : ['subject_id', 'hadm_id', 'stay_id', 'value', 'label', 'delta', 'uom', 't_0']
gcs : ['subject_id', 'hadm_id', 'stay_id', 'day', 'mingcs', 'gcsmotor', 'gcsverbal', 'gcseyes', 'endotrachflag', 'icu_admit', 't_0', 'approx_charttime', 'admit_plus_day', 'delta', 'uom']
heightfirstday : ['stay_id', 'height', 'uom', 't_0']
labs_all_nosummary : ['subject_id', 'hadm_id', 'stay_id', 'charttime', 'label', 'valuenum', 'uom', 't_0', 'delta']
pt_info : ['stay_id', 'subject_id', 't_0', 'label', 'value', 'delta', 'uom']
rrt_merged : ['stay_id', 't_0', 'first_charttime', 'rrt', 'uom']
sum_elix : ['subject_id', 'hadm_id', 'stay_id', 'value', 'label', 'delta', 'uom', 't_0']
vaso_dose : ['stay_id', 'linkorderid', 'vaso_rate', 'vaso_amount', 'charttime', 'endtime', 'amount_uom', 'rate_uom', 'label', 't_0', 'delta']
ventcategory : ['stay_id', 'day', 'valu

In [61]:
from parameters import time_var, value_fill, delta_fill, uom_fill
def yn_convert(df, #df in format where each row corresponds to a test, and a patient can have many rows
               label_fill, # value that will be filled to na's
               pt= final_pt_df2,
               time_var=time_var,#'t_0', #
               value_fill=value_fill,#0,
               delta_fill=delta_fill,# pd.to_timedelta('0 days'),
               uom_fill=uom_fill):#'y/n'):

    """
    description: collapses (binarizes) a dataframe where each row corresponds to a test, and a patient can have many rows -> 
    1 row per patient where value is binary variable yes or no a patient has any value within the timewindow (specified in data collection).
    said a different way, for patient this fxn collapses values down to does pt have a non NA value in the clinical time window y/n? 
    
    label_fill: the variable name in the label column of the specified dataframe that will considered for y/n value within timewindow. if any non NA value is present
        it will be considered positive.
    pt: the by patient spreadsheet be be used to supply patient information.
    time_var: the variable used to create the time window of interest.
    value_fill: the variable value that missing values will be filled if the value is not present (default =0) in the origional dataset
    delta_fill: the time delta value that will be filled in if a patient doesn't have any instances of the label_fill.  
    uom_fill: fills in the unit of measurement to this for missing values.
    
    returns a flat 1 row per stay_id of 1 or 0 if any value was present for the patient.
    """
    
    yn_df = pd.merge(pt[['stay_id', time_var]],
                      df[['stay_id','value','label','uom','delta']],
                     left_on= 'stay_id',
                     right_on= 'stay_id',
                      how='left') #merging all stay_id's with time_var, where value,label,uom, and delta are nan's if no value exists for that icustay. 
    #the idea is that if any value exists then it is pos.

    yn_df['value']= yn_df['value'].fillna(value_fill) #converts na to 0 in above na rows.
    yn_df.loc[yn_df.loc[:,'value']!=value_fill, 'value']= 1 #squashes all other values into a binary 1 = yes
    yn_df['delta']= yn_df['delta'].fillna(delta_fill)
    yn_df['delta']= pd.to_timedelta(yn_df['delta']) #filling in the time delta to time =0 for filled rows
    yn_df['uom']= yn_df['uom'].fillna(uom_fill)
    yn_df.loc[yn_df.loc[:,'uom']!=uom_fill, 'uom']= uom_fill
    yn_df['label']= yn_df['label'].fillna(label_fill)
    ##this is new as of 12-12-19L i think i wasn't truely converting to 1row per icustay id by not droping duplicates filtered on value first (desc) then delta (asc)
    yn_df= yn_df.sort_values(['value','delta'], ascending=[False, True]).drop_duplicates(subset='stay_id',keep='first')
    
    return(yn_df)


time: 15 ms (started: 2023-07-26 18:02:13 +08:00)


## vaso dose

In [64]:
# #renaming starttime to charttime and dropping endtime

dfs[vaso_df]= dfs[vaso_df].rename(
    columns={'starttime':'charttime','label':'vaso_type'})

dfs[vaso_df] = dfs[vaso_df].drop(['linkorderid'], axis = 1)
dfs[vaso_df] = dfs[vaso_df].drop(['endtime'], axis = 1)

time: 0 ns (started: 2023-07-26 18:04:35 +08:00)


In [65]:
dfs[vaso_df]

,stay_id,vaso_rate,vaso_amount,charttime,amount_uom,rate_uom,vaso_type,t_0,delta
0,30009339,0.010000,0.048000,2145-05-10 07:00:00,mg,mcg/kg/min,epinephrine,2145-05-10 04:00:00,0 days 03:00:00
1,30009339,0.100000,0.960800,2145-05-10 10:00:00,mg,mcg/kg/min,epinephrine,2145-05-10 04:00:00,0 days 06:00:00
5,30009339,1.007658,8.000001,2145-05-10 07:00:00,mg,mcg/kg/min,norepinephrine,2145-05-10 04:00:00,0 days 03:00:00
6,30009339,0.100000,0.654400,2145-05-10 10:00:00,mg,mcg/kg/min,norepinephrine,2145-05-10 04:00:00,0 days 06:00:00
9,30009339,2.496198,58.200008,2145-05-11 02:07:00,mg,mcg/kg/min,phenylephrine,2145-05-10 04:00:00,0 days 22:07:00
...,...,...,...,...,...,...,...,...,...
82251,39990190,0.300109,4.760656,2153-10-06 13:21:00,mg,mcg/kg/min,phenylephrine,2153-10-06 12:00:00,0 days 01:21:00
82252,39990190,0.250091,2.032787,2153-10-06 15:33:00,mg,mcg/kg/min,phenylephrine,2153-10-06 12:00:00,0 days 03:33:00
82253,39999552,0.200007,0.670824,2186-07-17 16:46:00,mg,mcg/kg/min,phenylephrine,2186-07-17 14:00:00,0 days 02:46:00
82254,39999552,0.300075,1.064516,2186-07-17 19:30:00,mg,mcg/kg/min,phenylephrine,2186-07-17 14:00:00,0 days 05:30:00


time: 16 ms (started: 2023-07-26 18:04:39 +08:00)


In [66]:
len(dfs[vaso_df])

22683

time: 0 ns (started: 2023-07-26 18:05:18 +08:00)


In [68]:
#removing units/hour because that is a different use of vasopressin
dfs[vaso_df] = dfs[vaso_df].loc[dfs[vaso_df].loc[:,'rate_uom']!= 'units/hour',:]
dfs[vaso_df] = dfs[vaso_df].loc[dfs[vaso_df].loc[:,'rate_uom']!= 'Uhr',:]

time: 0 ns (started: 2023-07-26 18:05:29 +08:00)


In [69]:
len(dfs[vaso_df])

22683

time: 16 ms (started: 2023-07-26 18:05:30 +08:00)


In [70]:
dfs[vaso_df]['rate_uom'].unique()

array(['mcg/kg/min', 'units/min', 'mg/kg/min'], dtype=object)

time: 0 ns (started: 2023-07-26 18:05:32 +08:00)


#### removing outliers/extreme values

In [74]:
# Use transform to add a column back to the orig df from a groupby aggregation, transform returns a Series with its index aligned to the orig df:
def vaso_outlier_removal(df):
    test_group=(dfs[vaso_df][['vaso_rate','rate_uom','amount_uom','vaso_type']]#.groupby('vaso_type', as_index=False)
         .groupby(['vaso_type','rate_uom','amount_uom'])
             )

    dfs[vaso_df]['std']=test_group.transform(lambda x : x.std())
    dfs[vaso_df]['mean']=test_group.transform(lambda x : x.mean())

    normal_high_value= pd.DataFrame({
        'vaso_type' : ['dobutamine','dopamine','epinephrine','norepinephrine','vasopressin','phenylephrine'],
        'high_value': [40, 20, 0.5, 1, 0.1, 2] #highest values one might expect to see in a clinic, ie above this is likely erroneous
    }) #found from literature, see notes

    dfs[vaso_df] = pd.merge(dfs[vaso_df], normal_high_value, left_on='vaso_type', right_on='vaso_type')

    vaso_dose_72_rmout =(dfs[vaso_df][
        ~((dfs[vaso_df]['vaso_rate'] > dfs[vaso_df]['high_value']) & ((dfs[vaso_df]['vaso_rate']-dfs[vaso_df]['mean'])>= (3*dfs[vaso_df]['std'])))
        ])
    #ie vaso_dose_72_rmout is a dataframe of all rows that excludes rows where vaso rate > literature high value and where vaso_rate >3sd from teh mean
    return(vaso_dose_72_rmout)

time: 0 ns (started: 2023-07-26 18:10:33 +08:00)


In [75]:
dfs[vaso_df]= vaso_outlier_removal(dfs[vaso_df])
len(dfs[vaso_df]) #22683 ->22607

22607

time: 47 ms (started: 2023-07-26 18:10:35 +08:00)


In [76]:
dfs[vaso_df]['vaso_type'].unique()

array(['epinephrine', 'norepinephrine', 'phenylephrine', 'dopamine',
       'dobutamine', 'vasopressin'], dtype=object)

time: 15 ms (started: 2023-07-26 18:10:59 +08:00)


In [77]:
#standardizing names, dropping unneeded columns for analysis
dfs[vaso_df]= dfs[vaso_df].drop(['vaso_amount', 'amount_uom','std','mean','high_value'], axis=1)
dfs[vaso_df]= dfs[vaso_df].rename(index=str, columns={'vaso_rate': 'value', 'rate_uom':'uom','vaso_type':'label'})
dfs[vaso_df]['label'].unique()

array(['epinephrine', 'norepinephrine', 'phenylephrine', 'dopamine',
       'dobutamine', 'vasopressin'], dtype=object)

time: 16 ms (started: 2023-07-26 18:11:04 +08:00)


In [78]:
dfs[vaso_df].head()

,stay_id,value,charttime,uom,label,t_0,delta
0,30009339,0.010000,2145-05-10 07:00:00,mcg/kg/min,epinephrine,2145-05-10 04:00:00,0 days 03:00:00
1,30009339,0.100000,2145-05-10 10:00:00,mcg/kg/min,epinephrine,2145-05-10 04:00:00,0 days 06:00:00
2,30029746,0.020000,2126-10-26 11:45:00,mcg/kg/min,epinephrine,2126-10-26 11:00:00,0 days 00:45:00
3,30029746,0.010003,2126-10-27 04:47:00,mcg/kg/min,epinephrine,2126-10-26 11:00:00,0 days 17:47:00
4,30049088,0.030191,2198-03-28 04:46:00,mcg/kg/min,epinephrine,2198-03-27 20:00:00,0 days 08:46:00


time: 0 ns (started: 2023-07-26 18:11:09 +08:00)


In [79]:
epinephrine_df= dfs[vaso_df][dfs[vaso_df]['label']=='epinephrine']
norepinephrine_df= dfs[vaso_df][dfs[vaso_df]['label']=='norepinephrine']
phenylephrine_df= dfs[vaso_df][dfs[vaso_df]['label']=='phenylephrine']
vasopressin_df= dfs[vaso_df][dfs[vaso_df]['label']=='vasopressin']
dopamine_df= dfs[vaso_df][dfs[vaso_df]['label']=='dopamine']
dobutamine_df= dfs[vaso_df][dfs[vaso_df]['label']=='dobutamine']


#making a all vasoactive
all_vaso=pd.concat([epinephrine_df, norepinephrine_df,phenylephrine_df, vasopressin_df,dopamine_df,dobutamine_df ], sort=False)#.sort_values(['stay_id','delta','label','source'], ascending=True)
#sorting so that a patient's most positive and earliest cases are kept and everything else droped. 
all_vaso= all_vaso.sort_values(['value','delta'], ascending=[False, True]).drop_duplicates(subset='stay_id',keep='first')
all_vaso['label']='any_vasoactives'


time: 31 ms (started: 2023-07-26 18:11:31 +08:00)


In [80]:
#(could be transfered to a different spdsheet for collapsing values)
#y/n convert, seperating out vaso_dose into 6 constitutient dataframes, and for each am collapsing values down to does pt have in time window y/n? 

epinephrine_df=yn_convert(epinephrine_df, label_fill='epinephrine', pt= final_pt_df2, value_fill=0, delta_fill=0, uom_fill='y/n', time_var=time_var)
norepinephrine_df=yn_convert(norepinephrine_df, label_fill='norepinephrine', pt= final_pt_df2, value_fill=0, delta_fill=0, uom_fill='y/n', time_var=time_var)
phenylephrine_df=yn_convert(phenylephrine_df, label_fill='phenylephrine', pt= final_pt_df2, value_fill=0, delta_fill=0, uom_fill='y/n', time_var=time_var)
vasopressin_df=yn_convert(vasopressin_df, label_fill='vasopressin', pt= final_pt_df2, value_fill=0, delta_fill=0, uom_fill='y/n', time_var=time_var)
dopamine_df=yn_convert(dopamine_df, label_fill='dopamine', pt= final_pt_df2, value_fill=0, delta_fill=0, uom_fill='y/n', time_var=time_var)
dobutamine_df=yn_convert(dobutamine_df, label_fill='dobutamine', pt= final_pt_df2, value_fill=0, delta_fill=0, uom_fill='y/n', time_var=time_var)
all_vaso=yn_convert(all_vaso, label_fill='any_vasoactives', pt= final_pt_df2, value_fill=0, delta_fill=0, uom_fill='y/n', time_var=time_var)

time: 750 ms (started: 2023-07-26 18:11:35 +08:00)


In [82]:
norepinephrine_df

,stay_id,t_0,value,label,uom,delta
682,35121579,2131-06-12 18:00:00,1.0,norepinephrine,y/n,0 days
686,39527567,2129-05-09 08:00:00,1.0,norepinephrine,y/n,0 days
1238,35977849,2126-07-20 22:00:00,1.0,norepinephrine,y/n,0 days
1679,36692638,2190-12-21 00:00:00,1.0,norepinephrine,y/n,0 days
2674,31528057,2154-10-11 03:00:00,1.0,norepinephrine,y/n,0 days
...,...,...,...,...,...,...
32699,37692584,2197-07-26 08:00:00,0.0,norepinephrine,y/n,0 days
32700,35165301,2197-08-04 20:00:00,0.0,norepinephrine,y/n,0 days
32701,37364566,2162-08-16 08:00:00,0.0,norepinephrine,y/n,0 days
32702,36075953,2149-01-08 20:00:00,0.0,norepinephrine,y/n,0 days


time: 0 ns (started: 2023-07-26 18:11:54 +08:00)


In [84]:
norepinephrine_df[norepinephrine_df['delta'] != '0 days']

,stay_id,t_0,value,label,uom,delta
340,37631738,2184-05-10 02:00:00,1.0,norepinephrine,y/n,0 days 00:01:00
9701,34393102,2182-12-05 06:00:00,1.0,norepinephrine,y/n,0 days 00:01:00
11338,36125394,2176-04-09 16:00:00,1.0,norepinephrine,y/n,0 days 00:01:00
11559,36391674,2187-06-20 16:00:00,1.0,norepinephrine,y/n,0 days 00:01:00
14957,38645374,2142-11-10 22:00:00,1.0,norepinephrine,y/n,0 days 00:01:00
...,...,...,...,...,...,...
21916,38110624,2183-03-20 21:00:00,1.0,norepinephrine,y/n,0 days 23:59:00
5918,31024755,2126-04-25 17:00:00,1.0,norepinephrine,y/n,1 days 00:00:00
20275,35756924,2137-01-19 11:00:00,1.0,norepinephrine,y/n,1 days 00:00:00
31038,32594563,2171-01-28 15:00:00,1.0,norepinephrine,y/n,1 days 00:00:00


time: 16 ms (started: 2023-07-26 18:13:02 +08:00)


In [85]:
all_vaso['stay_id'].nunique() #26471

26471

time: 15 ms (started: 2023-07-26 18:13:30 +08:00)


In [86]:
#all_vaso=all_vaso.sort_values(['value','delta'], ascending=[False, True]).drop_duplicates(subset='stay_id',keep='first')
len(all_vaso)


26471

time: 16 ms (started: 2023-07-26 18:13:42 +08:00)


In [87]:
save_df(all_vaso, df_name='all_vaso')
del(all_vaso)

save_df(epinephrine_df, df_name='epinephrine')
del(epinephrine_df)

save_df(norepinephrine_df, df_name='norepinephrine')
del(norepinephrine_df)

save_df(phenylephrine_df, df_name='phenylephrine')
del(phenylephrine_df)

save_df(vasopressin_df, df_name='vasopressin')
del(vasopressin_df)

save_df(dopamine_df, df_name='dopamine')
del(dopamine_df)

save_df(dobutamine_df, df_name='dobutamine')
del(dobutamine_df)

del(dfs[vaso_df])


c:\Users\csfhe\Documents\RA\mimiciii-antibiotics-opensource\notebooks/data/cleaned/24_hr_window/
time: 3.2 s (started: 2023-07-26 18:13:49 +08:00)


## ELIX

In [88]:
#convert cancer elix to y/n:
dfs[cancer_elix_df]= yn_convert(dfs[cancer_elix_df], label_fill=0, pt= final_pt_df2, value_fill=0, delta_fill=0, uom_fill='y/n', time_var=time_var)
save_df(dfs[cancer_elix_df], df_name='cancer_elix')
del(dfs[cancer_elix_df])

save_df(dfs[sum_elix_df], 'sum_elix')
del(dfs[sum_elix_df])

time: 734 ms (started: 2023-07-26 18:14:04 +08:00)


## vitals -

In [89]:
dfs[vitals_df].head()

,subject_id,hadm_id,stay_id,charttime,valueuom,vitalid,valuenum,t_0,delta
77,10001884,26184834,37510196,2131-01-11 11:00:00,insp/min,RespRate,20.0,2131-01-11 08:00:00,0 days 03:00:00
78,10001884,26184834,37510196,2131-01-11 16:00:00,insp/min,RespRate,19.0,2131-01-11 08:00:00,0 days 08:00:00
305,10001884,26184834,37510196,2131-01-12 07:00:00,insp/min,RespRate,18.0,2131-01-11 08:00:00,0 days 23:00:00
384,10001884,26184834,37510196,2131-01-12 07:00:00,°C,TempC,36.7,2131-01-11 08:00:00,0 days 23:00:00
385,10001884,26184834,37510196,2131-01-12 07:00:00,bpm,HeartRate,85.0,2131-01-11 08:00:00,0 days 23:00:00


time: 16 ms (started: 2023-07-26 18:14:12 +08:00)


In [90]:
dfs[vitals_df].loc[:,'vitalid'].unique()

array(['RespRate', 'TempC', 'HeartRate', 'SpO2', 'SysBP', 'DiasBP',
       'MeanArtPress', 'fio2_chartevents', 'TempF', 'Glucose', nan],
      dtype=object)

time: 187 ms (started: 2023-07-26 18:14:36 +08:00)


In [91]:
dfs[bg_df].loc[dfs[bg_df].loc[:,'label']=='fio2_chartevents',:]#unique()

,subject_id,hadm_id,stay_id,charttime,label,valuenum,value,valueuom,unique_var,t_0,delta


time: 31 ms (started: 2023-07-26 18:14:54 +08:00)


In [92]:
dfs[vitals_df] = dfs[vitals_df].rename(index=str, columns={"valueuom":"uom","vitalid":'label', 'valuenum':'value'}) #change valueom to uom
dfs[vitals_df] = dfs[vitals_df].loc[dfs[vitals_df]['label'].notnull(),:]#.count() #removing null values


dfs[vitals_df].loc[dfs[vitals_df].loc[:,'uom']=='BPM','uom']='bpm'

#overall the values are extremely similar and are likely the same thing
#i will combine them.
dfs[vitals_df].loc[
    (dfs[vitals_df]['label']=='RespRate') & 
    (dfs[vitals_df]['uom']=='bpm'),'uom']='insp/min'

dfs[vitals_df].loc[
    (dfs[vitals_df]['label']=='TempC') & 
    (dfs[vitals_df]['uom']=='?C'),'uom']='Deg. C'

dfs[vitals_df].loc[
    (dfs[vitals_df]['label']=='TempF') & 
    (dfs[vitals_df]['uom']=='Deg. F'),'uom']='Deg. C'

dfs[vitals_df].loc[
    (dfs[vitals_df]['label']=='TempF') & 
    (dfs[vitals_df]['uom']=='?F'),'uom']='Deg. C'

dfs[vitals_df].loc[
    (dfs[vitals_df]['label']=='TempF'),'label']='temperature'

dfs[vitals_df].loc[
    (dfs[vitals_df]['label']=='TempC'),'label']='temperature'

time: 4.92 s (started: 2023-07-26 18:15:01 +08:00)


- glucose max
- glucose min
- diasBP min
- heartrate min
- meanart pressure min
- RespRate min
- SYSbp min
- TEMPC min

### most likely erroneous value removal

In [93]:
#erroneous value cutoff summary
## setting a conservative threshold for erroneous values to not skew my data.

(dfs[vitals_df].loc[(dfs[vitals_df].loc[:,'stay_id']==228393.0) &
                         (dfs[vitals_df].loc[:,'label']=='Glucose') &
                         (dfs[vitals_df].loc[:,'value']>99999), 'value'])=np.nan

(dfs[vitals_df].loc[(dfs[vitals_df].loc[:,'label']=='Glucose') &
                         (dfs[vitals_df].loc[:,'value']>99998), 'value'])=np.nan

(dfs[vitals_df].loc[(dfs[vitals_df].loc[:,'label']=='Glucose') &
                         (dfs[vitals_df].loc[:,'value']<15), 'value'])=np.nan

(dfs[vitals_df].loc[(dfs[vitals_df].loc[:,'label']=='DiasBP') &
                         (dfs[vitals_df].loc[:,'value']<15), 'value'])=np.nan

(dfs[vitals_df].loc[(dfs[vitals_df].loc[:,'label']=='HeartRate') &
                         (dfs[vitals_df].loc[:,'value'].between(1,29)), 'value'])=np.nan

(dfs[vitals_df].loc[(dfs[vitals_df].loc[:,'label']=='RespRate') &
                         (dfs[vitals_df].loc[:,'value']<4), 'value'])=np.nan

(dfs[vitals_df].loc[(dfs[vitals_df].loc[:,'label']=='SysBP') &
                         (dfs[vitals_df].loc[:,'value']<40), 'value'])=np.nan

(dfs[vitals_df].loc[(dfs[vitals_df].loc[:,'label']=='TempC') &
                         (dfs[vitals_df].loc[:,'value']<28), 'value'])=np.nan

time: 2.03 s (started: 2023-07-26 18:15:12 +08:00)


In [94]:
dfs[vitals_df] = dfs[vitals_df].loc[dfs[vitals_df]['value'].notnull(),:]#.count() 


time: 406 ms (started: 2023-07-26 18:15:17 +08:00)


In [95]:
### saving spo2 for later use in bloodgas 11/25/19
spo2=dfs[vitals_df][dfs[vitals_df]['label']=='SpO2'].copy()

### seperating fio2 out
fio2_chart_df = dfs[vitals_df].loc[dfs[vitals_df].loc[:,'label']=='fio2_chartevents',:]#unique()
dfs[vitals_df]= dfs[vitals_df].loc[dfs[vitals_df].loc[:,'label']!='fio2_chartevents',:]

time: 1.22 s (started: 2023-07-26 18:15:28 +08:00)


In [96]:
save_df(dfs[vitals_df], 'vitals')
del(dfs[vitals_df])


time: 19.3 s (started: 2023-07-26 18:15:31 +08:00)


# labs -

In [97]:
dfs[labs_df].head(10)

,subject_id,hadm_id,stay_id,charttime,label,valuenum,uom,t_0,delta
6,10001884,26184834,37510196,2131-01-12 03:34:00,HEMATOCRIT,33.6,%,2131-01-11 08:00:00,0 days 19:34:00
27,10001884,26184834,37510196,2131-01-12 03:34:00,HEMOGLOBIN,10.4,g/dL,2131-01-11 08:00:00,0 days 19:34:00
47,10001884,26184834,37510196,2131-01-12 03:34:00,PLATELET,149.0,K/uL,2131-01-11 08:00:00,0 days 19:34:00
66,10001884,26184834,37510196,2131-01-12 03:34:00,WBC,12.0,K/uL,2131-01-11 08:00:00,0 days 19:34:00
85,10001884,26184834,37510196,2131-01-12 03:34:00,ANION GAP,11.0,mEq/L,2131-01-11 08:00:00,0 days 19:34:00
103,10001884,26184834,37510196,2131-01-12 03:34:00,BICARBONATE,33.0,mEq/L,2131-01-11 08:00:00,0 days 19:34:00
118,10001884,26184834,37510196,2131-01-12 03:34:00,BILIRUBIN,0.2,mg/dL,2131-01-11 08:00:00,0 days 19:34:00
133,10001884,26184834,37510196,2131-01-12 03:34:00,CHLORIDE,98.0,mEq/L,2131-01-11 08:00:00,0 days 19:34:00
151,10001884,26184834,37510196,2131-01-12 03:34:00,CREATININE,1.0,mg/dL,2131-01-11 08:00:00,0 days 19:34:00
170,10001884,26184834,37510196,2131-01-12 03:34:00,GLUCOSE,145.0,mg/dL,2131-01-11 08:00:00,0 days 19:34:00


time: 0 ns (started: 2023-07-26 18:15:53 +08:00)


In [98]:
dfs[labs_df]= dfs[labs_df].rename(
    columns={'valuenum':'value'}) #changing valuenum to value

time: 47 ms (started: 2023-07-26 18:16:10 +08:00)


In [99]:
dfs[labs_df].groupby('label')['uom'].value_counts() #looks good

label        uom   
ALBUMIN      g/dL       8745
ANION GAP    mEq/L     51361
BANDS        %          2720
BICARBONATE  mEq/L     51680
BILIRUBIN    mg/dL     17672
BUN          mg/dL     51742
CHLORIDE     mEq/L     58868
CREATININE   mg/dL     51894
GLUCOSE      mg/dL     69153
HEMATOCRIT   %         64277
HEMOGLOBIN   g/dL      57896
LACTATE      mmol/L    45957
PLATELET     K/uL      50159
POTASSIUM    mEq/L     71601
PT           sec       34849
PTT          sec       36267
SODIUM       mEq/L     61871
WBC          K/uL      49041
Name: count, dtype: int64

time: 172 ms (started: 2023-07-26 18:16:11 +08:00)


### most likely erroneous value removal

In [100]:
#summary value removal- was explored and coded adhoc, difficult to automate
(dfs[labs_df].loc[
    (dfs[labs_df].loc[:,'stay_id']==261887) & 
    (dfs[labs_df].loc[:,'label']=='CHLORIDE')
    & (dfs[labs_df].loc[:,'value']==3.4),'value'])=np.nan

(dfs[labs_df].loc[
    (dfs[labs_df].loc[:,'stay_id']==236290) & 
    (dfs[labs_df].loc[:,'label']=='CHLORIDE')
    & (dfs[labs_df].loc[:,'value']==11.0),'value'])=np.nan

(dfs[labs_df].loc[
    (dfs[labs_df].loc[:,'stay_id']==292769) & 
    (dfs[labs_df].loc[:,'label']=='INR')
    & (dfs[labs_df].loc[:,'value']==28.1),'value'])=np.nan

(dfs[labs_df].loc[
    (dfs[labs_df].loc[:,'stay_id']==298457) & 
    (dfs[labs_df].loc[:,'label']=='INR')
    & (dfs[labs_df].loc[:,'value']==48.8),'value'])=np.nan

(dfs[labs_df].loc[
    (dfs[labs_df].loc[:,'stay_id']==234174) & 
    (dfs[labs_df].loc[:,'label']=='INR')
    & (dfs[labs_df].loc[:,'value']==48.7),'value'])=np.nan

(dfs[labs_df].loc[
    (dfs[labs_df].loc[:,'stay_id']==290264) & 
    (dfs[labs_df].loc[:,'label']=='INR')
    & (dfs[labs_df].loc[:,'value']==42.0),'value'])=np.nan

(dfs[labs_df].loc[
    (dfs[labs_df].loc[:,'stay_id']==290264) & 
    (dfs[labs_df].loc[:,'label']=='INR')
    & (dfs[labs_df].loc[:,'value']==22.8),'value'])=np.nan

time: 344 ms (started: 2023-07-26 18:16:31 +08:00)


In [101]:
dfs[labs_df]= dfs[labs_df].loc[dfs[labs_df].loc[:,'value'].notnull(),:] #removing null values


time: 78 ms (started: 2023-07-26 18:16:41 +08:00)


In [102]:
dfs[labs_df].head()

,subject_id,hadm_id,stay_id,charttime,label,value,uom,t_0,delta
6,10001884,26184834,37510196,2131-01-12 03:34:00,HEMATOCRIT,33.6,%,2131-01-11 08:00:00,0 days 19:34:00
27,10001884,26184834,37510196,2131-01-12 03:34:00,HEMOGLOBIN,10.4,g/dL,2131-01-11 08:00:00,0 days 19:34:00
47,10001884,26184834,37510196,2131-01-12 03:34:00,PLATELET,149.0,K/uL,2131-01-11 08:00:00,0 days 19:34:00
66,10001884,26184834,37510196,2131-01-12 03:34:00,WBC,12.0,K/uL,2131-01-11 08:00:00,0 days 19:34:00
85,10001884,26184834,37510196,2131-01-12 03:34:00,ANION GAP,11.0,mEq/L,2131-01-11 08:00:00,0 days 19:34:00


time: 0 ns (started: 2023-07-26 18:16:44 +08:00)


In [103]:
#removing unwanted values
unwanted_values= ['HEMATOCRIT','ANION GAP','PT','ALBUMIN']
dfs[labs_df]= dfs[labs_df].loc[~dfs[labs_df].loc[:,'label'].isin(unwanted_values),:]

time: 78 ms (started: 2023-07-26 18:16:50 +08:00)


# factorizing bands start 
* converting bands into a categorical variable since it is very sparse

In [104]:

#df in format where each row corresponds to a test, and a patient can have many rows
def yn_convert_band(df,           
                    label_fill="absent",
                    threshold=10,
                    pt= final_pt_df2,
                    time_var='t_0',
                    value_fill=9999,
                    delta_fill=pd.to_timedelta('0 days'),
                    uom_fill='y/n'):
    
    """
    binarizing bands value since it's extremely sparse. Wrote a new function instead of adjusting yn_convert a while back. on my todo to remove this and add functionality to regular yn_conmert function. 
    """
    
    yn_df = pd.merge(pt[['stay_id','hadm_id','subject_id', time_var]],
                      df[['stay_id','value','label','uom','delta']],
                     left_on= 'stay_id',
                     right_on= 'stay_id',
                      how='left') #merging all stay_id's with time_var, where value,label,uom, and delta are nan's if no value exists for that icustay. 
    #the idea is that if any value exists then it is pos.

    yn_df['value']= yn_df['value'].fillna(value_fill) #converts na to 0 in above na rows.
    criteria0=yn_df.loc[:,'value']==value_fill
    criteria1=pd.to_numeric(yn_df.loc[:,'value'])<=threshold
    criteria2=pd.to_numeric(yn_df.loc[:,'value'])>threshold


    yn_df.loc[criteria1, 'value']= "<{}".format(threshold) 
    yn_df.loc[criteria2, 'value']= ">{}".format(threshold) 
    yn_df.loc[criteria0, 'value']= "absent"

    yn_df['delta']= yn_df['delta'].fillna(delta_fill)
    yn_df['delta']= pd.to_timedelta(yn_df['delta']) #filling in the time delta to time =0 for filled rows
    yn_df['uom']= yn_df['uom'].fillna(uom_fill)
    yn_df.loc[yn_df.loc[:,'uom']!=uom_fill, 'uom']= uom_fill
    yn_df['label']= yn_df['label'].fillna(label_fill)
    
    return(yn_df)
    

time: 0 ns (started: 2023-07-26 18:17:05 +08:00)


In [106]:
band_df

,subject_id,hadm_id,stay_id,charttime,label,value,uom,t_0,delta
2519,10003400,23559586,38383343,2137-08-18 04:38:00,BANDS,1.0,%,2137-08-17 20:00:00,0 days 08:38:00
7173,10007818,22987108,32359580,2146-06-22 14:05:00,BANDS,3.0,%,2146-06-22 12:00:00,0 days 02:05:00
14541,10017531,21095812,32136798,2159-12-27 04:39:00,BANDS,3.0,%,2159-12-26 23:00:00,0 days 05:39:00
20073,10021927,24623461,34575919,2180-09-21 00:55:00,BANDS,1.0,%,2180-09-20 13:00:00,0 days 11:55:00
21426,10023486,25262533,31361200,2151-06-09 23:35:00,BANDS,2.0,%,2151-06-09 22:00:00,0 days 01:35:00
...,...,...,...,...,...,...,...,...,...
8127122,19970491,25338284,38969862,2129-09-16 03:25:00,BANDS,5.0,%,2129-09-15 18:00:00,0 days 09:25:00
8142676,19985293,21731208,34896989,2184-08-20 04:25:00,BANDS,11.0,%,2184-08-20 04:00:00,0 days 00:25:00
8144627,19987071,22919294,33385977,2113-12-28 04:31:00,BANDS,23.0,%,2113-12-27 23:00:00,0 days 05:31:00
8146102,19990427,29695607,34058245,2182-01-14 09:04:00,BANDS,1.0,%,2182-01-14 05:00:00,0 days 04:04:00


time: 16 ms (started: 2023-07-26 18:18:06 +08:00)


In [115]:
band_df=dfs[labs_df][dfs[labs_df]['label']=='BANDS']
max_bands=band_df.loc[band_df.groupby('stay_id', as_index=False)['value'].idxmax()['value'],:]
del(band_df)

time: 265 ms (started: 2023-07-26 18:25:01 +08:00)


In [116]:
band_cat=yn_convert_band(df=max_bands,           
                    label_fill="BANDS",
                    threshold=10,
                    pt= final_pt_df2,
                    time_var='t_0',
                    value_fill=9999,
                    delta_fill=pd.to_timedelta('0 days'),
                    uom_fill='y/n')

time: 141 ms (started: 2023-07-26 18:25:05 +08:00)


In [117]:
band_cat

,stay_id,hadm_id,subject_id,t_0,value,label,uom,delta
0,37510196,26184834,10001884,2131-01-11 08:00:00,absent,BANDS,y/n,0 days
1,33685454,23822395,10002155,2129-08-05 14:00:00,absent,BANDS,y/n,0 days
2,31090461,28994087,10002155,2130-09-25 11:00:00,absent,BANDS,y/n,0 days
3,32358465,20345487,10002155,2131-03-10 01:00:00,absent,BANDS,y/n,0 days
4,38875437,28662225,10002428,2156-04-20 12:00:00,absent,BANDS,y/n,0 days
...,...,...,...,...,...,...,...,...
26466,37692584,22997012,19999287,2197-07-26 08:00:00,absent,BANDS,y/n,0 days
26467,35165301,20175828,19999287,2197-08-04 20:00:00,absent,BANDS,y/n,0 days
26468,37364566,21439025,19999297,2162-08-16 08:00:00,absent,BANDS,y/n,0 days
26469,36075953,25744818,19999828,2149-01-08 20:00:00,absent,BANDS,y/n,0 days


time: 16 ms (started: 2023-07-26 18:25:20 +08:00)


In [119]:
#drop bands from lab_df
dfs[labs_df]=dfs[labs_df].drop(dfs[labs_df][dfs[labs_df]['label']=='BANDS'].index)

##dropping charttime, may be problematic later.
#dfs[labs_df]=dfs[labs_df].drop('charttime', axis=1)

#appending categorical bands 
dfs[labs_df]=dfs[labs_df]._append(band_cat)

time: 219 ms (started: 2023-07-26 18:27:25 +08:00)


In [120]:
dfs[labs_df].loc[dfs[labs_df]['label']=='BANDS','value'].value_counts()
# absent    24174
# <10        1630
# >10         667

value
absent    24174
<10        1630
>10         667
Name: count, dtype: int64

time: 46 ms (started: 2023-07-26 18:27:30 +08:00)


In [121]:
# pd.DataFrame(dfs[labs_df]).to_csv(Path(
#     save_path+'/%s_labs_prepped.csv' %(date)))

save_df(dfs[labs_df], 'labs')
del(dfs[labs_df])

time: 3.78 s (started: 2023-07-26 18:28:05 +08:00)


## vent category -

In [233]:
dfs[vent_df]['stay_id'].nunique()
#13978 patients with someform of vent data. 

25728

time: 0 ns (started: 2023-07-26 19:10:56 +08:00)


In [234]:
dfs[vent_df]['uom']='mech/O2/none category'
dfs[vent_df].head()

,stay_id,day,value,uom,t_0
0,30000484,0,Oxygen,mech/O2/none category,2136-01-14 21:00:00
1,30000484,1,NaN,mech/O2/none category,2136-01-14 21:00:00
2,30000646,0,Oxygen,mech/O2/none category,2194-04-29 07:00:00
3,30001446,0,Oxygen,mech/O2/none category,2186-04-12 01:00:00
4,30001555,0,Oxygen,mech/O2/none category,2177-09-27 11:00:00


time: 15 ms (started: 2023-07-26 19:10:58 +08:00)


In [235]:
dfs[vent_df]=dfs[vent_df].rename(index=str, columns={'day':'delta'})
dfs[vent_df]['label']='vent_recieved'
dfs[vent_df]['delta']=pd.to_timedelta(dfs[vent_df]['delta'], unit='d')
#dfs[vent_df]= dfs[vent_df].drop(columns=['day'], axis=1) #removing day column

time: 16 ms (started: 2023-07-26 19:11:01 +08:00)


In [236]:
dfs[vent_df].head()

,stay_id,delta,value,uom,t_0,label
0,30000484,0 days,Oxygen,mech/O2/none category,2136-01-14 21:00:00,vent_recieved
1,30000484,1 days,NaN,mech/O2/none category,2136-01-14 21:00:00,vent_recieved
2,30000646,0 days,Oxygen,mech/O2/none category,2194-04-29 07:00:00,vent_recieved
3,30001446,0 days,Oxygen,mech/O2/none category,2186-04-12 01:00:00,vent_recieved
4,30001555,0 days,Oxygen,mech/O2/none category,2177-09-27 11:00:00,vent_recieved


time: 0 ns (started: 2023-07-26 19:11:03 +08:00)


In [58]:
#collapsing into 1 column for N days

time: 579 µs


In [237]:
def vent_day_collapser(x):
    """
    collapsing the ventilation days into a single value. Mech> Oxygen > None. 
    """
    
    if 'Mech' in list(x.unique()):
        x= 'Mech'
    elif 'Oxygen' in list(x.unique()):
        x= 'Oxygen'
    else:
        x='None'
    return(x)

time: 0 ns (started: 2023-07-26 19:11:05 +08:00)


In [238]:
#collapsing all days into the worst day. 
ventcategory_1day_df= dfs[vent_df].copy()
ventcategory_1day_df['value']=ventcategory_1day_df.groupby('stay_id',as_index=False)['value'].transform(vent_day_collapser)
ventcategory_1day_df= ventcategory_1day_df.drop_duplicates(['stay_id','value']).sort_values('stay_id') #
ventcategory_1day_df= ventcategory_1day_df.loc[ventcategory_1day_df.loc[:,'stay_id'].isin(stay_id),:] #had icustay ids not in final cohort, fail safe mesure
ventcategory_1day_df.head()

,stay_id,delta,value,uom,t_0,label
0,30000484,0 days,Oxygen,mech/O2/none category,2136-01-14 21:00:00,vent_recieved
2,30000646,0 days,Oxygen,mech/O2/none category,2194-04-29 07:00:00,vent_recieved
3,30001446,0 days,Oxygen,mech/O2/none category,2186-04-12 01:00:00,vent_recieved
4,30001555,0 days,Oxygen,mech/O2/none category,2177-09-27 11:00:00,vent_recieved
5,30002415,0 days,Mech,mech/O2/none category,2126-12-17 12:00:00,vent_recieved


time: 3.73 s (started: 2023-07-26 19:11:07 +08:00)


In [128]:
ventcategory_1day_df['value'].value_counts()
# value
# Mech      12330
# Oxygen    10114
# None       3284

value
Mech      12330
Oxygen    10114
None       3284
Name: count, dtype: int64

time: 0 ns (started: 2023-07-26 18:29:00 +08:00)


In [129]:
save_df(ventcategory_1day_df, 'ventcategory')
# will be deleted after pao2:fio2 calc

time: 453 ms (started: 2023-07-26 18:29:20 +08:00)


## weight and height firstday -
i explored weightdurations and it had more missing values than weightfirstday, so i will use that. we can revisit this if we need longitudinal weights

In [130]:
dfs[weight_df]['uom']='kg'
dfs[weight_df].head()  
#weight column is the conglomerate of weight_admin>weight_daily> weight_echoinhosp> weight_echoprehosp

,stay_id,weight,weight_admit,weight_daily,uom,t_0
0,30000484,68.5,68.5,NaN,kg,2136-01-14 21:00:00
1,30000646,NaN,NaN,NaN,kg,2194-04-29 07:00:00
2,30001446,119.3,119.3,NaN,kg,2186-04-12 01:00:00
3,30001555,NaN,NaN,NaN,kg,2177-09-27 11:00:00
4,30002415,81.0,81.0,86.0,kg,2126-12-17 12:00:00


time: 16 ms (started: 2023-07-26 18:29:31 +08:00)


In [131]:
dfs[weight_df]= dfs[weight_df][dfs[weight_df]['weight'].notnull()]
dfs[weight_df]= dfs[weight_df][['stay_id','weight','uom']]

dfs[weight_df]['label']= 'weight'
dfs[weight_df]=dfs[weight_df].rename(index=str, columns={'weight':'value'})

#adding the assumed first day delta column to standardize all columns
dfs[weight_df]['delta']=pd.to_timedelta(0,'days')

#adding t_0
dfs[weight_df]= pd.merge(dfs[weight_df], final_pt_df2[['stay_id',time_var]], left_on='stay_id', right_on='stay_id')

time: 16 ms (started: 2023-07-26 18:29:34 +08:00)


In [132]:
# pd.DataFrame(dfs[weight_df]).to_csv(Path(
#     save_path+'/%s_weight_prepped.csv' %(date)))
save_df(dfs[weight_df], 'weight')

del(dfs[weight_df])

time: 407 ms (started: 2023-07-26 18:29:36 +08:00)


In [133]:
dfs[height_df]['uom']='cm'

time: 0 ns (started: 2023-07-26 18:29:38 +08:00)


In [134]:
dfs[height_df]= dfs[height_df][dfs[height_df]['height'].notnull()]
dfs[height_df]= dfs[height_df][['stay_id','height','uom']]

time: 0 ns (started: 2023-07-26 18:29:39 +08:00)


In [135]:
dfs[height_df]['label']= 'height'
dfs[height_df]=dfs[height_df].rename(index=str, columns={'height':'value'})

#adding the assumed first day delta column to standardize all columns
dfs[height_df]['delta']=pd.to_timedelta(0,'days')
#adding t_0
dfs[height_df]= pd.merge(dfs[height_df], final_pt_df2[['stay_id',time_var]], left_on='stay_id', right_on='stay_id')

time: 16 ms (started: 2023-07-26 18:29:41 +08:00)


In [136]:
#heightfirstday
save_df(dfs[height_df], 'height')
del(dfs[height_df])

time: 234 ms (started: 2023-07-26 18:29:43 +08:00)


# UTI

In [70]:
dfs[uti_df]['value'].unique()#seems good #all uti within clinical timewindow

array(['NEG', 'MOD', 'POS', 'TR', 'SM', 'LG', ' ', 'N', 'SMALL'],
      dtype=object)

time: 13.1 ms


In [71]:
dfs[uti_df]= dfs[uti_df].loc[(dfs[uti_df].loc[:,'value']!='NEG')&
               dfs[uti_df].loc[:,'value'].notnull(),:] #filter to only pos rows

dfs[uti_df]= dfs[uti_df].loc[(dfs[uti_df].loc[:,'value']!='COMPUTER NETWORK FAILURE. TEST NOT RESULTED.')&
               dfs[uti_df].loc[:,'value'].notnull(),:] #filter to only pos rows
dfs[uti_df]= dfs[uti_df].drop_duplicates(subset=['hadm_id','value','charttime'])

dfs[uti_df].loc[dfs[uti_df].loc[:,'value'].notnull(),'value']= 1
dfs[uti_df].loc[dfs[uti_df].loc[:,'value'].isna(),'value']= 0 

time: 189 ms


In [72]:
dfs[uti_df].head()

,subject_id,hadm_id,itemid,charttime,value,valuenum,valueuom,label,fluid,category,loinc_code,icd9_code,t_0,delta
132,21,109451.0,51486,2134-09-12 00:08:00,1,NaN,NaN,Leukocytes,Urine,Hematology,5799-2,41071,2134-09-12,0 days 00:08:00.000000000
186,21,109451.0,51487,2134-09-12 00:08:00,1,NaN,NaN,Nitrite,Urine,Hematology,5802-4,41071,2134-09-12,0 days 00:08:00.000000000
236,31,128652.0,51486,2108-08-24 16:30:00,1,NaN,NaN,Leukocytes,Urine,Hematology,5799-2,3453,2108-08-24,0 days 16:30:00.000000000
790,64,172056.0,51486,2143-03-03 04:33:00,1,NaN,NaN,Leukocytes,Urine,Hematology,5799-2,03811,2143-03-03,0 days 04:33:00.000000000
832,62,116009.0,51486,2113-02-15 12:27:00,1,NaN,NaN,Leukocytes,Urine,Hematology,5799-2,1125,2113-02-15,0 days 12:27:00.000000000


time: 18.9 ms


In [73]:
def uti_categorizer(uti_df):
    "useful to get all rows of days with positive values for patients (if multiple pos in a day there will be only 1 row for that day). ie more longitudinal format "
    
    #gives the max pos or neg value per day for 
    df_timewindow_perday=uti_df.groupby(['hadm_id','delta'], as_index=False)['value'].agg({'value':'max'})  
    df_timewindow_perday= pd.merge(df_timewindow_perday,final_pt_df2[['stay_id','hadm_id', time_var]], left_on='hadm_id', right_on='hadm_id', how='left')
    df_timewindow_perday=df_timewindow_perday.sort_values(['hadm_id','value','delta'], ascending=[True,False,True])
    return(df_timewindow_perday)

def yn_uti(uti_df, label):
    "collapsing longitudinal data into 1 value. will return pos or neg if patient has a positive uti in their stay. one row per stay_id"
    
    df_timewindow_perday=uti_df.groupby(['hadm_id','delta'], as_index=False)['value'].agg({'value':'max'})
    first_pos= df_timewindow_perday.drop_duplicates(['hadm_id'])
    collapsed= pd.merge(final_pt_df2[['hadm_id','stay_id','subject_id', time_var]],first_pos, left_on='hadm_id', right_on='hadm_id', how='left')
    
    collapsed['value']= collapsed['value'].fillna(0)
    collapsed.loc[collapsed.loc[:,'value']==1,'value']= 'pos'
    collapsed.loc[collapsed.loc[:,'value']==0,'value']= 'Neg/Not_tested'
    
    collapsed['delta']= collapsed['delta'].fillna(pd.Timedelta(1, unit='d'))
    
    collapsed['label']= label
    
    collapsed['uom']='pos/neg category'
    return(collapsed)

time: 73.8 ms


In [74]:
uti_nit_pos= dfs[uti_df][dfs[uti_df]['label']=="Nitrite"]
uti_leuk_pos= dfs[uti_df][dfs[uti_df]['label']=="Leukocytes"]

leuk_collapsed= yn_uti(uti_leuk_pos, 'leukocyte')
nit_collapsed= yn_uti(uti_nit_pos, 'nitrite')


time: 135 ms


In [75]:
save_df(leuk_collapsed, 'leuk')
del(leuk_collapsed)

save_df(nit_collapsed, 'nit')
del(nit_collapsed)
del(dfs[uti_df])

time: 587 ms


# bloodgas

In [206]:
dfs[bg_df]

,subject_id,hadm_id,stay_id,charttime,label,valuenum,value,valueuom,unique_var,t_0,delta
46,10001884,26184834,37510196,2131-01-11 11:33:00,BASEEXCESS,2.00,2,mEq/L,375101962131-01-11 11:33:00,2131-01-11 08:00:00,0 days 03:33:00
47,10001884,26184834,37510196,2131-01-11 11:33:00,LACTATE,1.10,1.1,mmol/L,375101962131-01-11 11:33:00,2131-01-11 08:00:00,0 days 03:33:00
48,10001884,26184834,37510196,2131-01-11 11:33:00,PCO2,59.00,59,mm Hg,375101962131-01-11 11:33:00,2131-01-11 08:00:00,0 days 03:33:00
49,10001884,26184834,37510196,2131-01-11 11:33:00,PH,7.33,7.33,units,375101962131-01-11 11:33:00,2131-01-11 08:00:00,0 days 03:33:00
50,10001884,26184834,37510196,2131-01-11 11:33:00,PO2,40.00,___,mm Hg,375101962131-01-11 11:33:00,2131-01-11 08:00:00,0 days 03:33:00
...,...,...,...,...,...,...,...,...,...,...,...
2782800,19999840,21033226,38978960,2164-09-13 06:26:00,BASEEXCESS,4.00,4,mEq/L,389789602164-09-13 06:26:00,2164-09-12 20:00:00,0 days 10:26:00
2782801,19999840,21033226,38978960,2164-09-13 06:26:00,PCO2,37.00,37,mm Hg,389789602164-09-13 06:26:00,2164-09-12 20:00:00,0 days 10:26:00
2782802,19999840,21033226,38978960,2164-09-13 06:26:00,PH,7.50,7.50,units,389789602164-09-13 06:26:00,2164-09-12 20:00:00,0 days 10:26:00
2782803,19999840,21033226,38978960,2164-09-13 06:26:00,PO2,61.00,61,mm Hg,389789602164-09-13 06:26:00,2164-09-12 20:00:00,0 days 10:26:00


time: 15 ms (started: 2023-07-26 19:08:10 +08:00)


In [207]:
dfs[bg_df]= dfs[bg_df].loc[dfs[bg_df]['value'].notnull(),:]
dfs[bg_df] = dfs[bg_df].rename(index=str, columns={'valueuom':'uom'})
dfs[bg_df]= dfs[bg_df].loc[~(dfs[bg_df].loc[:,'value']=='.'),:]

#may need to remove outliers, haven't done as of 10/22/18
dfs[bg_df].head()

,subject_id,hadm_id,stay_id,charttime,label,valuenum,value,uom,unique_var,t_0,delta
46,10001884,26184834,37510196,2131-01-11 11:33:00,BASEEXCESS,2.00,2,mEq/L,375101962131-01-11 11:33:00,2131-01-11 08:00:00,0 days 03:33:00
47,10001884,26184834,37510196,2131-01-11 11:33:00,LACTATE,1.10,1.1,mmol/L,375101962131-01-11 11:33:00,2131-01-11 08:00:00,0 days 03:33:00
48,10001884,26184834,37510196,2131-01-11 11:33:00,PCO2,59.00,59,mm Hg,375101962131-01-11 11:33:00,2131-01-11 08:00:00,0 days 03:33:00
49,10001884,26184834,37510196,2131-01-11 11:33:00,PH,7.33,7.33,units,375101962131-01-11 11:33:00,2131-01-11 08:00:00,0 days 03:33:00
50,10001884,26184834,37510196,2131-01-11 11:33:00,PO2,40.00,___,mm Hg,375101962131-01-11 11:33:00,2131-01-11 08:00:00,0 days 03:33:00


time: 407 ms (started: 2023-07-26 19:08:11 +08:00)


### most likely erroneous value removal
the code below manually removes values that are a high likelyhood of being erroneous in a way that doesn't follow great programming practices. on my list of things to improve, but it is currently functional. 


In [208]:
##calcium
#fixing the calcium errors w/o hard coding
(dfs[bg_df].loc[
    (dfs[bg_df].loc[:,'stay_id']==249571) & 
    (dfs[bg_df].loc[:,'label']=='CALCIUM')
    & (dfs[bg_df].loc[:,'valuenum']==94.00),'valuenum'])=0.94#.where('valuenum'==94.00))

(dfs[bg_df].loc[
    (dfs[bg_df].loc[:,'stay_id']==249571) & 
    (dfs[bg_df].loc[:,'label']=='CALCIUM')
    & (dfs[bg_df].loc[:,'value']=='094'),'value'])=0.94#.where('valuenum'==94.00))


(dfs[bg_df].loc[
    (dfs[bg_df].loc[:,'stay_id']==219600) & 
    (dfs[bg_df].loc[:,'label']=='CALCIUM')
    & (dfs[bg_df].loc[:,'valuenum']==97.00),'valuenum'])=0.97#.where('valuenum'==94.00))

(dfs[bg_df].loc[
    (dfs[bg_df].loc[:,'stay_id']==219600) & 
    (dfs[bg_df].loc[:,'label']=='CALCIUM')
    & (dfs[bg_df].loc[:,'value']=='097'),'value'])=0.97#.where('valuenum'==94.00))


time: 203 ms (started: 2023-07-26 19:08:13 +08:00)


In [209]:
##min chloride
#converting it to a null value without hard coding
(dfs[bg_df].loc[(dfs[bg_df].loc[:,'stay_id']==261887.0) &
                         (dfs[bg_df].loc[:,'label']=='CHLORIDE') &
                         (dfs[bg_df].loc[:,'valuenum']==3.4),'value'])=np.nan
#converting it to a null value without hard coding
(dfs[bg_df].loc[(dfs[bg_df].loc[:,'stay_id']==261887.0) &
                         (dfs[bg_df].loc[:,'label']=='CHLORIDE') &
                         (dfs[bg_df].loc[:,'valuenum']==3.4),'valuenum'])=np.nan

#changing the values without hard coding. 
(dfs[bg_df].loc[(dfs[bg_df].loc[:,'stay_id']==236290.0) &
                         (dfs[bg_df].loc[:,'label']=='CHLORIDE') &
                         (dfs[bg_df].loc[:,'valuenum']==11.0),'valuenum'])=np.nan

(dfs[bg_df].loc[(dfs[bg_df].loc[:,'stay_id']==236290.0) &
                         (dfs[bg_df].loc[:,'label']=='CHLORIDE') &
                         (dfs[bg_df].loc[:,'valuenum']==11.0),'value'])=np.nan

time: 125 ms (started: 2023-07-26 19:08:15 +08:00)


In [210]:
#peep changes summary: 
dfs[bg_df].loc[(dfs[bg_df].loc[:,'label']=='PEEP')&
                        (dfs[bg_df]['valuenum']>38),'valuenum']=np.nan #remove this or set to 50?

time: 31 ms (started: 2023-07-26 19:08:17 +08:00)


In [211]:
#temp changes summary:

dfs[bg_df].loc[(dfs[bg_df].loc[:,'label']=='TEMPERATURE')&
                        (dfs[bg_df]['stay_id']==253821)&
                        (dfs[bg_df]['valuenum']==18.9),
                        'value']= np.nan

dfs[bg_df].loc[(dfs[bg_df].loc[:,'label']=='TEMPERATURE')&
                        (dfs[bg_df]['stay_id']==253821)&
                        (dfs[bg_df]['valuenum']==18.9),
                        'valuenum']= np.nan

dfs[bg_df].loc[(dfs[bg_df].loc[:,'label']=='TEMPERATURE')&
                        (dfs[bg_df]['stay_id']==251788)&
                        (dfs[bg_df]['valuenum']==10.0),
                        'value']= np.nan

dfs[bg_df].loc[(dfs[bg_df].loc[:,'label']=='TEMPERATURE')&
                        (dfs[bg_df]['stay_id']==251788)&
                        (dfs[bg_df]['valuenum']==10.0),
                        'valuenum']= np.nan

time: 125 ms (started: 2023-07-26 19:08:19 +08:00)


In [212]:
#fio2 changes summary:

##converting a few values to null, thus removing them from the dataset
dfs[bg_df].loc[(dfs[bg_df].loc[:,'label']=='FIO2') &
                        (dfs[bg_df].loc[:,'value']=='0'),'value']=np.nan

dfs[bg_df].loc[(dfs[bg_df].loc[:,'label']=='FIO2') &
                        (dfs[bg_df].loc[:,'value']=='-'),'value']=np.nan


##removing all fio2 values between 1-20.9

dfs[bg_df].loc[(dfs[bg_df].loc[:,'label'].isin(['FIO2']))&
                        (dfs[bg_df]['valuenum'].between(1.0,20.9)),'valuenum']=np.nan


##values between 0-1 were found to be ratios, not %, so converting these to % 

dfs[bg_df].loc[(dfs[bg_df].loc[:,'label'].isin(['FIO2']))&
                        (dfs[bg_df]['valuenum'].between(1.0,20.9)),'valuenum']=np.nan

fio2_dec= dfs[bg_df].loc[(dfs[bg_df].loc[:,'label'].isin(['FIO2']))&
                        (dfs[bg_df]['valuenum'].between(0.0,1.0)),'valuenum']

dfs[bg_df].loc[(dfs[bg_df].loc[:,'label'].isin(['FIO2']))&
                        (dfs[bg_df]['valuenum'].between(0.0,1.0)),'valuenum'] = fio2_dec *100

del(fio2_dec)

time: 156 ms (started: 2023-07-26 19:08:21 +08:00)


#### removing null values annotated abov

In [213]:
dfs[bg_df]= dfs[bg_df].loc[dfs[bg_df]['value'].notnull(),:]

time: 63 ms (started: 2023-07-26 19:08:24 +08:00)


In [214]:
dfs[bg_df]

,subject_id,hadm_id,stay_id,charttime,label,valuenum,value,uom,unique_var,t_0,delta
46,10001884,26184834,37510196,2131-01-11 11:33:00,BASEEXCESS,2.00,2,mEq/L,375101962131-01-11 11:33:00,2131-01-11 08:00:00,0 days 03:33:00
47,10001884,26184834,37510196,2131-01-11 11:33:00,LACTATE,1.10,1.1,mmol/L,375101962131-01-11 11:33:00,2131-01-11 08:00:00,0 days 03:33:00
48,10001884,26184834,37510196,2131-01-11 11:33:00,PCO2,59.00,59,mm Hg,375101962131-01-11 11:33:00,2131-01-11 08:00:00,0 days 03:33:00
49,10001884,26184834,37510196,2131-01-11 11:33:00,PH,7.33,7.33,units,375101962131-01-11 11:33:00,2131-01-11 08:00:00,0 days 03:33:00
50,10001884,26184834,37510196,2131-01-11 11:33:00,PO2,40.00,___,mm Hg,375101962131-01-11 11:33:00,2131-01-11 08:00:00,0 days 03:33:00
...,...,...,...,...,...,...,...,...,...,...,...
2782800,19999840,21033226,38978960,2164-09-13 06:26:00,BASEEXCESS,4.00,4,mEq/L,389789602164-09-13 06:26:00,2164-09-12 20:00:00,0 days 10:26:00
2782801,19999840,21033226,38978960,2164-09-13 06:26:00,PCO2,37.00,37,mm Hg,389789602164-09-13 06:26:00,2164-09-12 20:00:00,0 days 10:26:00
2782802,19999840,21033226,38978960,2164-09-13 06:26:00,PH,7.50,7.50,units,389789602164-09-13 06:26:00,2164-09-12 20:00:00,0 days 10:26:00
2782803,19999840,21033226,38978960,2164-09-13 06:26:00,PO2,61.00,61,mm Hg,389789602164-09-13 06:26:00,2164-09-12 20:00:00,0 days 10:26:00


time: 0 ns (started: 2023-07-26 19:08:25 +08:00)


### splitting specimen out for vent vs non-vent bg data

In [ ]:
#adding specimen tag to filter only arterial samples for vent data. 
specimen_df= dfs[bg_df].loc[dfs[bg_df].loc[:,'label']=='SPECIMEN',['unique_var','label','value']]#unique()
specimen_df=specimen_df.rename(index=str, columns={'value':'specimen'})
specimen_df=specimen_df.loc[specimen_df.loc[:,"specimen"]=='ART',:]

dfs[bg_df]= pd.merge(dfs[bg_df],specimen_df[['unique_var','specimen']], left_on='unique_var', right_on='unique_var', how='left')
bg_ART_nosummary=dfs[bg_df].loc[dfs[bg_df].loc[:,'specimen']=='ART',:].copy()
del(specimen_df)
#I guess mimiciv does not need this step

In [215]:
bg_labels=['PH','LACTATE','CALCIUM','TEMPERATURE','POTASSIUM',
           'GLUCOSE','HEMOGLOBIN','SODIUM','CHLORIDE','BICARBONATE','FIO2']
bg_vent_labels=['PCO2','PaO2','PO2','PEEP','O2FLOW']

time: 0 ns (started: 2023-07-26 19:08:28 +08:00)


In [216]:
bg_ART_nosummary= dfs[bg_df].loc[dfs[bg_df].loc[:,'label'].isin(bg_vent_labels),:]

time: 32 ms (started: 2023-07-26 19:08:29 +08:00)


In [217]:
#restricting to tests that were chosen to be analysed based on %missingness and clinical relevance
dfs[bg_df]= dfs[bg_df].loc[dfs[bg_df].loc[:,'label'].isin(bg_labels),:]


time: 47 ms (started: 2023-07-26 19:08:32 +08:00)


In [218]:
dfs[bg_df]

,subject_id,hadm_id,stay_id,charttime,label,valuenum,value,uom,unique_var,t_0,delta
47,10001884,26184834,37510196,2131-01-11 11:33:00,LACTATE,1.10,1.1,mmol/L,375101962131-01-11 11:33:00,2131-01-11 08:00:00,0 days 03:33:00
49,10001884,26184834,37510196,2131-01-11 11:33:00,PH,7.33,7.33,units,375101962131-01-11 11:33:00,2131-01-11 08:00:00,0 days 03:33:00
55,10001884,26184834,37510196,2131-01-11 14:48:00,PH,7.36,7.36,units,375101962131-01-11 14:48:00,2131-01-11 08:00:00,0 days 06:48:00
59,10001884,26184834,37510196,2131-01-11 22:21:00,FIO2,50.00,50,%,375101962131-01-11 22:21:00,2131-01-11 08:00:00,0 days 14:21:00
61,10001884,26184834,37510196,2131-01-11 22:21:00,PH,7.32,7.32,units,375101962131-01-11 22:21:00,2131-01-11 08:00:00,0 days 14:21:00
...,...,...,...,...,...,...,...,...,...,...,...
2782767,19999828,25744818,36075953,2149-01-08 20:04:00,PH,7.36,7.36,units,360759532149-01-08 20:04:00,2149-01-08 20:00:00,0 days 00:04:00
2782772,19999828,25744818,36075953,2149-01-09 14:22:00,PH,7.35,7.35,units,360759532149-01-09 14:22:00,2149-01-08 20:00:00,0 days 18:22:00
2782777,19999828,25744818,36075953,2149-01-09 19:58:00,PH,7.35,7.35,units,360759532149-01-09 19:58:00,2149-01-08 20:00:00,0 days 23:58:00
2782795,19999840,21033226,38978960,2164-09-13 02:49:00,PH,7.53,7.53,units,389789602164-09-13 02:49:00,2164-09-12 20:00:00,0 days 06:49:00


time: 15 ms (started: 2023-07-26 19:08:33 +08:00)


In [219]:
bg_ART_nosummary

,subject_id,hadm_id,stay_id,charttime,label,valuenum,value,uom,unique_var,t_0,delta
48,10001884,26184834,37510196,2131-01-11 11:33:00,PCO2,59.0,59,mm Hg,375101962131-01-11 11:33:00,2131-01-11 08:00:00,0 days 03:33:00
50,10001884,26184834,37510196,2131-01-11 11:33:00,PO2,40.0,___,mm Hg,375101962131-01-11 11:33:00,2131-01-11 08:00:00,0 days 03:33:00
54,10001884,26184834,37510196,2131-01-11 14:48:00,PCO2,64.0,___,mm Hg,375101962131-01-11 14:48:00,2131-01-11 08:00:00,0 days 06:48:00
56,10001884,26184834,37510196,2131-01-11 14:48:00,PO2,46.0,___,mm Hg,375101962131-01-11 14:48:00,2131-01-11 08:00:00,0 days 06:48:00
60,10001884,26184834,37510196,2131-01-11 22:21:00,PCO2,67.0,___,mm Hg,375101962131-01-11 22:21:00,2131-01-11 08:00:00,0 days 14:21:00
...,...,...,...,...,...,...,...,...,...,...,...
2782793,19999840,21033226,38978960,2164-09-13 02:49:00,PCO2,32.0,32,mm Hg,389789602164-09-13 02:49:00,2164-09-12 20:00:00,0 days 06:49:00
2782794,19999840,21033226,38978960,2164-09-13 02:49:00,PEEP,5.0,5,NaN,389789602164-09-13 02:49:00,2164-09-12 20:00:00,0 days 06:49:00
2782796,19999840,21033226,38978960,2164-09-13 02:49:00,PO2,86.0,86,mm Hg,389789602164-09-13 02:49:00,2164-09-12 20:00:00,0 days 06:49:00
2782801,19999840,21033226,38978960,2164-09-13 06:26:00,PCO2,37.0,37,mm Hg,389789602164-09-13 06:26:00,2164-09-12 20:00:00,0 days 10:26:00


time: 15 ms (started: 2023-07-26 19:08:41 +08:00)


In [220]:
bg_col=['subject_id','hadm_id','stay_id','charttime','delta',time_var,'label','valuenum','uom']
dfs[bg_df]= dfs[bg_df][bg_col]

time: 0 ns (started: 2023-07-26 19:08:44 +08:00)


In [221]:
bg_ART_nosummary= bg_ART_nosummary[bg_col]

time: 0 ns (started: 2023-07-26 19:08:46 +08:00)


In [222]:
del(bg_col)

time: 0 ns (started: 2023-07-26 19:08:47 +08:00)


In [223]:
dfs[bg_df]['label'].unique()

array(['LACTATE', 'PH', 'FIO2', 'TEMPERATURE', 'CALCIUM', 'HEMOGLOBIN',
       'GLUCOSE', 'CHLORIDE', 'POTASSIUM', 'SODIUM', 'BICARBONATE'],
      dtype=object)

time: 15 ms (started: 2023-07-26 19:08:48 +08:00)


In [224]:
bg_ART_nosummary['label'].unique()

array(['PCO2', 'PO2', 'PEEP', 'O2FLOW'], dtype=object)

time: 0 ns (started: 2023-07-26 19:08:50 +08:00)


In [89]:
##quickly investingating o2flow

time: 563 µs


In [225]:
o2_pt= list(dfs[vent_df].loc[dfs[vent_df].loc[:,'value']=='Oxygen','stay_id'].unique())

#o2_pt= list(ventcategory_1day_df.loc[ventcategory_1day_df.loc[:,'value']=='Oxygen','stay_id'].unique())

time: 0 ns (started: 2023-07-26 19:09:01 +08:00)


In [226]:
bg_ART_nosummary.loc[bg_ART_nosummary.loc[:,'label']=='O2FLOW',:]
#bg_ART_nosummary.loc[bg_ART_nosummary.loc[:,'stay_id']==217847,:]
bg_ART_nosummary.loc[(bg_ART_nosummary.loc[:,'label']=='O2FLOW') & (bg_ART_nosummary.loc[:,'stay_id'].isin(o2_pt)),:]


,subject_id,hadm_id,stay_id,charttime,delta,t_0,label,valuenum,uom
1468,10005817,28661809,31316840,2135-01-04 15:31:00,0 days 10:31:00,2135-01-04 05:00:00,O2FLOW,10.0,L/min
4244,10020306,23052851,38540883,2135-01-21 21:39:00,0 days 04:39:00,2135-01-21 17:00:00,O2FLOW,4.0,L/min
4253,10020306,23052851,38540883,2135-01-22 02:41:00,0 days 09:41:00,2135-01-21 17:00:00,O2FLOW,4.0,L/min
7927,10035168,25449821,39978210,2145-12-25 14:35:00,0 days 11:35:00,2145-12-25 03:00:00,O2FLOW,3.0,L/min
10119,10039708,28258130,33281088,2140-01-24 12:15:00,0 days 16:15:00,2140-01-23 20:00:00,O2FLOW,2.0,L/min
...,...,...,...,...,...,...,...,...,...
2749606,19882955,22092141,30779446,2194-09-05 01:33:00,0 days 00:33:00,2194-09-05 01:00:00,O2FLOW,2.0,L/min
2751848,19891107,26303115,36218320,2131-06-25 13:32:00,0 days 05:32:00,2131-06-25 08:00:00,O2FLOW,6.0,L/min
2753419,19899954,28627517,39480480,2115-02-07 07:14:00,0 days 06:14:00,2115-02-07 01:00:00,O2FLOW,2.0,L/min
2754336,19907884,20726042,35690576,2181-08-17 20:22:00,0 days 18:22:00,2181-08-17 02:00:00,O2FLOW,2.0,L/min


time: 31 ms (started: 2023-07-26 19:09:11 +08:00)


In [92]:
#dfs[vent_df].loc[dfs[vent_df].loc[:,'stay_id']==217847,:]


time: 812 µs


In [227]:
#converting O2 to y/n
o2_flow_df= bg_ART_nosummary.loc[bg_ART_nosummary.loc[:,'label']=='O2FLOW',:]
o2_flow_df=o2_flow_df.rename(index=str, columns={'valuenum':'value'})
o2_flow_df= yn_convert(o2_flow_df, label_fill='o2_flow', time_var=time_var)
o2_flow_df['label']="o2_flow" #fixing label

#removing o2_flow from bg_ART
bg_ART_nosummary= bg_ART_nosummary.loc[bg_ART_nosummary.loc[:,'label']!='O2FLOW',:]

time: 141 ms (started: 2023-07-26 19:09:16 +08:00)


In [228]:
bg_ART_nosummary= bg_ART_nosummary.loc[bg_ART_nosummary.loc[:,'label']!='PEEP',:]

time: 31 ms (started: 2023-07-26 19:09:18 +08:00)


## PaO2:FiO2
* PaO2: measurement of oxygen pressure in arterial blood
* FiO2: % of oxygen in the air a patient is breathing. in normal air this is 21% oxygen
* SpO2: Peripheral capillary oxygen saturation, estimate of the amount of oxygen in the blood. this is % of oxygenated haemoglobin compared to total haemoglobin

***requirements: run vitals, ventilation before this to get spo2 & ventilation category.
I would modify the criteria to be more strict:
* first isolate patients on mech ventilation: these are only patients we calc P:F for
* find PaO2:
    * if no PaO2:
     * find SpO2 <=97 --> ~PaO2
       * SpO2 to estimate and PaO2 equivalent via the equation used by Knox et al to convert to PaO2-equivalent.
* once have all PaO2 and estimated PaO2:
 * find last measured fio2 between t- 6hours :t, where t= time of PaO2  
* impute P:F= 476 for every other stay_id. 
* Use P:F as a continuous value unless there is a clear need to bucket, and if so use <100,100-200, 200-300, >300

##### first isolate patients who had mech or oxygen ventilation during their N hours
these are only patients we calc P:F for

first, filter pao2 and fio2 to only icustay who have ventilation (pf_ratio_icu)

In [229]:
vent_icu=list(ventcategory_1day_df[ventcategory_1day_df['value'].isin(['Mech'])]['stay_id'].unique())

time: 16 ms (started: 2023-07-26 19:09:22 +08:00)


next isolate the pao2 and fio2 from the bg arterial dataframe. 
 * pao2: all pao2 measurements in cohort -> pao2 measurements for icustay with ventilation
 * fio2: all fio2 measurements (chartevents) in cohort -> fio2 measurements for icustay with ventilation

In [230]:
#using bloodgas

fio2_chart_df['delta']=pd.to_timedelta(fio2_chart_df['delta'])
fio2_chart_df= fio2_chart_df[fio2_chart_df['value'].notnull()].copy()

pao2= bg_ART_nosummary.loc[bg_ART_nosummary.loc[:,'label']=='PaO2',:].copy() 
pao2.rename(index=str, columns={'valuenum':'value'}, inplace=True)
pao2_vent= pao2[pao2.loc[:,'stay_id'].isin(vent_icu)].copy()
pao2_vent['delta']=pd.to_timedelta(pao2_vent['delta']).copy()
pao2_vent= pao2_vent[pao2_vent['value'].notnull()] #removing some null rows

time: 219 ms (started: 2023-07-26 19:09:27 +08:00)


In [242]:
pao2

,subject_id,hadm_id,stay_id,charttime,delta,t_0,label,value,uom


time: 16 ms (started: 2023-07-26 19:12:34 +08:00)


##### next: find patients with ventilation but no PaO2 and collect their SpO2 values

In [231]:
# pao2_icu: stay_id for pao2 with vent
pao2_icu= list(pao2_vent['stay_id'].unique())

# find people with ventilation but no PaO2 measured (ie those who need it approximatd)
vent_but_no_pao2_icu=set(vent_icu).difference(set(pao2_icu))

## if on vent but no PaO2, then use SpO2 (if 97 or less) -> estimate PaO2 via equation below
spo2_filtered= spo2[(spo2['stay_id'].isin(vent_but_no_pao2_icu)) &
                    (spo2['value']<=97.0)]

time: 31 ms (started: 2023-07-26 19:09:31 +08:00)


In [232]:
##quick qc output
print(
    "total icustay:",  len(stay_id), '\n',
    'mech during their 72 hours of clinical data: ',  len(vent_icu), '\n',
    '   -pao2, +vent: ',  pao2_vent['stay_id'].nunique(), '\n',
    '   +fio2, +vent: ',  fio2_chart_df[fio2_chart_df['stay_id'].isin(vent_icu)]['stay_id'].nunique(), '\n',
    
    '+ vent, - pao2: ',  len(vent_but_no_pao2_icu), '\n',
    '+fio2, +vent, -pao2: ',  fio2_chart_df[fio2_chart_df['stay_id'].isin(vent_but_no_pao2_icu)]['stay_id'].nunique(), '\n',
)

total icustay: 26471 
 mech during their 72 hours of clinical data:  12330 
    -pao2, +vent:  0 
    +fio2, +vent:  12185 
 + vent, - pao2:  12330 
 +fio2, +vent, -pao2:  12185 

time: 31 ms (started: 2023-07-26 19:09:35 +08:00)


##### next: estimate PaO2 using SpO2 for patients who have ventilation but no PaO2 measurement 
$$\text{Ellis Severinghaus inversion}: PaO2_{estimate} =\sqrt[3]{\frac{1}{2} (-y_N + \sqrt{y_N^2 -h^2})} + \sqrt[3]{\frac{1}{2} (-y_N - \sqrt{y_N^2 -h^2})}$$

$$ \begin{cases} 
      h^2 =-500000 \\
      y_N = -23400 * \frac{s}{1-s} \\
   \end{cases}
$$
* s is the fractional oxygen saturation (0–1) and p is the associated oxygen tension in mm Hg. 
* qc: when s=0.5, p=26.856
* equation source: http://www.nickalls.org/dick/papers/anes/severinghaus.pdf


In [243]:
def Ellis_SpO2(S):
    """
    ellis inversion equation to approximate PaO2 from SpO2. 
    where s is the fractional oxygen saturation (SpO2) (0–1) and p is the associated oxygen tension in mm Hg (PaO2). 
    """
    if S >1 and S<100:
        S=S/100.
    elif S==1:
        S=0.9999
    elif S>0 and S<1:
        pass

    h2= -500000.
    yn= -23400. * (S/(1.-S))

    term1= np.sqrt(np.power(yn,2.) - h2)

    term2= 0.5*(-yn + term1)
    term3= 0.5*(-yn - term1)

    term4= np.sign(term2) * np.abs(term2)**(1/3)
    term5= np.sign(term3) * np.abs(term3)**(1/3)
    p= term4+term5
    return(p)
    

time: 0 ns (started: 2023-07-26 19:13:57 +08:00)


In [244]:
# approximating Pao2 from SpO2:

## only usable for spo2 values <=97
spo2_filtered= spo2[(spo2['stay_id'].isin(vent_but_no_pao2_icu)) &
                    (spo2['value']<=97.0)]

# run the Ellis_SpO2 to approx PaO2.
spo2_filtered.loc[:,'value']= spo2_filtered.value.apply(Ellis_SpO2) #note this produces a false positive SettingWithCopyWarning

spo2_filtered.loc[:,'label']='PaO2_estimate'
spo2_filtered.loc[:,'uom']='mm Hg estimate'


time: 1.06 s (started: 2023-07-26 19:14:00 +08:00)


In [246]:
#append the pao2 (for those with ventilation) and new approximated pao2 together
pao2_appended=pao2_vent._append(spo2_filtered, sort=False).copy()

time: 47 ms (started: 2023-07-26 19:14:09 +08:00)


* once have all PaO2 and estimated PaO2:
 * find fio2 measured between t- 6hours :t, where t= time of PaO2  
 * use most recent fio2 measure in this time?

In [247]:
#quick time conversion to make sure both in timedelta format
pao2_appended['delta']=pd.to_timedelta(pao2_appended['delta'])
pao2_appended['charttime']=pd.to_datetime(pao2_appended['charttime'])
fio2_chart_df['delta']=pd.to_timedelta(fio2_chart_df['delta'])
fio2_chart_df['charttime']=pd.to_datetime(fio2_chart_df['charttime'])
fio2_chart_df['label']='fio2'

#labeling all PaO2 (both measured and approx) with offset column= the delta time offset by the n hours of time we look back for Fio2 values
pao2_appended['offset']=pao2_appended['charttime'] - pd.to_timedelta(6, unit='h') #find most recent in this. 

time: 531 ms (started: 2023-07-26 19:14:11 +08:00)


In [248]:
#left cartesian product of pao2 and fio2 measurements. 
pao2_fio2_windowed= pd.merge(pao2_appended,
              fio2_chart_df[['stay_id','value','label','charttime','delta']].rename(index=str, columns={'value':'value2','label':'label2','charttime':'charttime2','delta':'delta2'}),
              left_on='stay_id',
              right_on='stay_id',
              how='left') 

##take this left cartesian product and filter to rows where charttime2 (ie fio2 time) is between t-6:t where t= time at pao2 measurement. 
#now this should be all Pao2 (real and estimated) and Fio2 combinations that 
pao2_fio2_windowed= pao2_fio2_windowed[pd.to_datetime(pao2_fio2_windowed['charttime2']).between(pd.to_datetime(pao2_fio2_windowed['offset']),pd.to_datetime(pao2_fio2_windowed['charttime']))]

time: 375 ms (started: 2023-07-26 19:14:13 +08:00)


In [249]:
## grabs the most recent FiO2 value for each associated PaO2 
ratio_df= pao2_fio2_windowed.sort_values(['stay_id','charttime','charttime2'], ascending=[True,True, False]).drop_duplicates(['stay_id','charttime'],keep='first')

time: 172 ms (started: 2023-07-26 19:14:14 +08:00)


In [250]:
ratio_df.loc[ratio_df.loc[:,'value2'].isnull(),'value2']= 21 #impute 21 for FiO2 values that are missing (very few of them)
ratio_df['ratio']= ratio_df['value']/(ratio_df['value2']/100) #calculate the ratio
ratio_df.loc[ratio_df['charttime2'].isnull(),'charttime2']= ratio_df.loc[ratio_df['charttime2'].isnull(),'charttime']
ratio_df['deltadelta']=pd.to_datetime(ratio_df['charttime'])-pd.to_datetime(ratio_df['charttime2'])

time: 31 ms (started: 2023-07-26 19:14:17 +08:00)


In [106]:
#optional qc
# ratio_df.head()
# ratio_df['ratio'].describe()
# ratio_df[['stay_id','ratio']].groupby('stay_id')['ratio'].apply(max).describe()

time: 1.1 ms


In [251]:
### making a copy of ratio_df, merging in all icustay's that are missing, and imputing datetime and value for them. 
ratio_df2= ratio_df.copy() # making sure all patients have a value
ratio_df2= pd.merge(final_pt_df2[['subject_id','hadm_id','stay_id']], ratio_df2[['stay_id', 'ratio','delta','uom','label']].rename(index=str, columns={'ratio':'value'}), left_on='stay_id',right_on='stay_id', how='outer')

ratio_df2['label']='pao2fio2ratio'
ratio_df2['uom']='mm HG:%'
ratio_df2.loc[ratio_df2['value'].isnull(),'value']=476
ratio_df2.loc[ratio_df2['delta'].isnull(),'delta']=pd.to_timedelta('0 days')

#should match the len(stay_id)
ratio_df2['stay_id'].nunique()

26471

time: 110 ms (started: 2023-07-26 19:14:19 +08:00)


In [252]:
ratio_df2.head()

,subject_id,hadm_id,stay_id,value,delta,uom,label
0,10001884,26184834,37510196,117.310871,0 days 04:22:00,mm HG:%,pao2fio2ratio
1,10001884,26184834,37510196,113.100695,0 days 07:49:00,mm HG:%,pao2fio2ratio
2,10001884,26184834,37510196,127.573449,0 days 09:06:00,mm HG:%,pao2fio2ratio
3,10001884,26184834,37510196,181.146106,0 days 21:00:00,mm HG:%,pao2fio2ratio
4,10002155,23822395,33685454,476.000000,0 days 00:00:00,mm HG:%,pao2fio2ratio


time: 0 ns (started: 2023-07-26 19:14:22 +08:00)


# factorizing pco2 start 

In [253]:
# yn_convert_band(max_bands, #df in format where each row corresponds to a test, and a patient can have many rows
def yn_convert_pco2(df,           
                    label_fill="absent",
                    threshold=10,
                    pt= final_pt_df2,
                    time_var='t_0',
                    value_fill=9999,
                    delta_fill=pd.to_timedelta('0 days'),
                    uom_fill='y/n'):
    
    yn_df = pd.merge(pt[['stay_id','hadm_id','subject_id', time_var]],
                      df[['stay_id','value','label','uom','delta']],
                     left_on= 'stay_id',
                     right_on= 'stay_id',
                      how='left') #merging all stay_id's with time_var, where value,label,uom, and delta are nan's if no value exists for that icustay. 
    #the idea is that if any value exists then it is pos.

    yn_df['value']= yn_df['value'].fillna(value_fill) #converts na to 0 in above na rows.
    criteria0=yn_df.loc[:,'value']==value_fill
    criteria1=pd.to_numeric(yn_df.loc[:,'value'])<=threshold
    criteria2=pd.to_numeric(yn_df.loc[:,'value'])>threshold


    yn_df.loc[criteria1, 'value']= "<{}".format(threshold) 
    yn_df.loc[criteria2, 'value']= ">{}".format(threshold) 
    yn_df.loc[criteria0, 'value']= "absent"

    yn_df['delta']= yn_df['delta'].fillna(delta_fill)
    yn_df['delta']= pd.to_timedelta(yn_df['delta']) #filling in the time delta to time =0 for filled rows
    yn_df['uom']= yn_df['uom'].fillna(uom_fill)
    yn_df.loc[yn_df.loc[:,'uom']!=uom_fill, 'uom']= uom_fill
    yn_df['label']= yn_df['label'].fillna(label_fill)
    
    return(yn_df)
    

time: 0 ns (started: 2023-07-26 19:14:26 +08:00)


In [254]:
bg_ART_nosummary= bg_ART_nosummary.rename(
    columns={'valuenum':'value'}) #changing valuenum to value

time: 0 ns (started: 2023-07-26 19:14:32 +08:00)


In [256]:
pco2_df=bg_ART_nosummary[bg_ART_nosummary['label']=='PCO2']
max_pco2=pco2_df.loc[pco2_df.groupby('stay_id', as_index=False)['value'].idxmax()['value'],:]
del(pco2_df)

time: 1.22 s (started: 2023-07-26 19:14:57 +08:00)


In [257]:
pco2_cat=yn_convert_pco2(df=max_pco2,           
                    label_fill="PCO2",
                    threshold=50,
                    pt= final_pt_df2,
                    time_var='t_0',
                    value_fill=9999,
                    delta_fill=pd.to_timedelta('0 days'),
                    uom_fill='y/n')

time: 140 ms (started: 2023-07-26 19:15:00 +08:00)


In [258]:
#drop bands from lab_df
bg_ART_nosummary=bg_ART_nosummary.drop(bg_ART_nosummary[bg_ART_nosummary['label']=='PCO2'].index)


time: 78 ms (started: 2023-07-26 19:15:02 +08:00)


In [259]:
##dropping charttime, may be problematic later. 06/13/19
bg_ART_nosummary=bg_ART_nosummary.drop('charttime', axis=1)

time: 15 ms (started: 2023-07-26 19:15:04 +08:00)


In [261]:
bg_ART_nosummary=bg_ART_nosummary._append(pco2_cat, sort=True)

time: 31 ms (started: 2023-07-26 19:15:10 +08:00)


In [262]:
bg_ART_nosummary.loc[bg_ART_nosummary['label']=='PCO2','value'].value_counts()
# value
# <50       12033
# absent    10194
# >50        4244

value
<50       12033
absent    10194
>50        4244
Name: count, dtype: int64

time: 16 ms (started: 2023-07-26 19:15:12 +08:00)


# factorizing bands end 

In [263]:
save_df(ratio_df2, 'pfRatio')
del( pao2_fio2_windowed,pao2_appended, fio2_chart_df, pao2, pao2_icu )
del( ventcategory_1day_df, dfs[vent_df] )

save_df(dfs[bg_df],'bg_all') #all bloodgas
del(dfs[bg_df])

save_df(bg_ART_nosummary,'bg_ART') #only ARTERIAL bloodgas
del(bg_ART_nosummary)

save_df(o2_flow_df,'o2_flow') #need to investigate this more
del(o2_flow_df)

time: 4.25 s (started: 2023-07-26 19:15:25 +08:00)


# RRT

In [264]:
#removing null values
dfs[rrt_df] = dfs[rrt_df].loc[dfs[rrt_df]['rrt'].notnull(),:]
dfs[rrt_df]['uom']='category'
dfs[rrt_df]['delta']=pd.to_timedelta(
    pd.to_datetime(dfs[rrt_df]['first_charttime'])-
    pd.to_datetime(dfs[rrt_df][time_var]),
    'days')
dfs[rrt_df]['label']= 'rrt'
dfs[rrt_df]=dfs[rrt_df].rename(index=str, columns={'rrt':'value'})

time: 47 ms (started: 2023-07-26 19:15:33 +08:00)


### converting to yes/no

In [265]:
dfs[rrt_df].head()

,stay_id,t_0,first_charttime,value,uom,delta,label
0,37510196,2131-01-11 08:00:00,NaN,0.0,category,NaT,rrt
1,33685454,2129-08-05 14:00:00,NaN,0.0,category,NaT,rrt
2,31090461,2130-09-25 11:00:00,NaN,0.0,category,NaT,rrt
3,32358465,2131-03-10 01:00:00,NaN,0.0,category,NaT,rrt
4,38875437,2156-04-20 12:00:00,NaN,0.0,category,NaT,rrt


time: 16 ms (started: 2023-07-26 19:15:35 +08:00)


In [266]:
dfs[rrt_df]['stay_id'].nunique() #26471

26471

time: 16 ms (started: 2023-07-26 19:15:38 +08:00)


In [267]:
rrt_yn= yn_convert(dfs[rrt_df],label_fill='rrt', time_var=time_var)

time: 32 ms (started: 2023-07-26 19:16:12 +08:00)


In [268]:
save_df(rrt_yn, 'rrt')
del(rrt_yn, dfs[rrt_df])

time: 532 ms (started: 2023-07-26 19:16:13 +08:00)


# GCS_72

In [269]:
dfs[gcs_df]['uom']='GCS_score'

time: 0 ns (started: 2023-07-26 19:16:28 +08:00)


In [270]:
list(dfs[gcs_df])

['subject_id',
 'hadm_id',
 'stay_id',
 'day',
 'mingcs',
 'gcsmotor',
 'gcsverbal',
 'gcseyes',
 'endotrachflag',
 'icu_admit',
 't_0',
 'approx_charttime',
 'admit_plus_day',
 'delta',
 'uom']

time: 0 ns (started: 2023-07-26 19:16:38 +08:00)


In [271]:
dfs[gcs_df]['label']= 'mingcs'
dfs[gcs_df]['uom']='gcs_score'
dfs[gcs_df]=dfs[gcs_df].rename(index=str, columns={'mingcs':'value'})

time: 15 ms (started: 2023-07-26 19:18:57 +08:00)


In [272]:
dfs[gcs_df]=dfs[gcs_df][['subject_id','hadm_id','stay_id','delta','label','value',time_var,'uom']]

time: 0 ns (started: 2023-07-26 19:18:58 +08:00)


In [273]:
save_df(dfs[gcs_df], 'gcs')
del(dfs[gcs_df])

time: 16 ms (started: 2023-07-26 19:19:00 +08:00)


# SOFA
i'm going to remove all sofa variables except daily score, as we have other markers for those in our above data
i may later use this as qc check.
also added delta

In [128]:
dfs[sofa_df]['uom']='daily_sofa_score'

#adding day delta column
dfs[sofa_df]=dfs[sofa_df].sort_values(['hadm_id','day',time_var]) #good
dfs[sofa_df]['day_rank']=dfs[sofa_df].groupby('stay_id')['day'].rank()
dfs[sofa_df]['delta']=pd.to_timedelta((dfs[sofa_df]['day_rank']-1), 'days')

time: 207 ms


In [129]:
dfs[sofa_df]['label']= 'daily_sofa'
dfs[sofa_df]=dfs[sofa_df].rename(index=str, columns={'sofa':'value'})

time: 221 ms


In [130]:
dfs[sofa_df]= dfs[sofa_df][['subject_id','hadm_id','stay_id','delta','label','value',time_var,'uom']]

time: 19.4 ms


In [131]:
dfs[sofa_df].head()

,subject_id,hadm_id,icustay_id,delta,label,value,t_0,uom
88189,54610,100003,209281,0 days,daily_sofa,4,2150-04-18,daily_sofa_score
52007,23018,100007,217937,0 days,daily_sofa,1,2145-03-31,daily_sofa_score
52008,23018,100007,217937,1 days,daily_sofa,4,2145-03-31,daily_sofa_score
52009,23018,100007,217937,2 days,daily_sofa,0,2145-03-31,daily_sofa_score
52010,23018,100007,217937,3 days,daily_sofa,3,2145-03-31,daily_sofa_score


time: 21.1 ms


In [132]:
save_df(dfs[sofa_df], 'sofa')
del(dfs[sofa_df])

time: 2.23 s


# patient Demographic variables

In [274]:
dfs[pt_info_df]

,stay_id,subject_id,t_0,label,value,delta,uom
0,37510196,10001884,2131-01-11 08:00:00,yearsold,68,0 days,yearsold
1,32358465,10002155,2131-03-10 01:00:00,yearsold,80,0 days,yearsold
2,33685454,10002155,2129-08-05 14:00:00,yearsold,80,0 days,yearsold
3,31090461,10002155,2130-09-25 11:00:00,yearsold,80,0 days,yearsold
4,36753294,10002495,2141-05-22 23:00:00,yearsold,81,0 days,yearsold
...,...,...,...,...,...,...,...
79408,36980198,19997367,2127-04-02 09:00:00,race,white/nonhispanic,0 days,NaN
79409,31417783,19998330,2178-10-22 18:00:00,race,black,0 days,NaN
79410,32641669,19998330,2178-10-02 13:00:00,race,black,0 days,NaN
79411,34403689,19998878,2132-10-01 00:00:00,race,white/nonhispanic,0 days,NaN


time: 15 ms (started: 2023-07-26 19:19:05 +08:00)


In [134]:
### gender distribution qc. should be almost entirely/entirely populated

time: 774 µs


In [275]:
#stay_id
#dfs[pt_info_df]
model_pts=list(final_pt_df2[final_pt_df2['final_bin'].isin(['C_pos/A_full','C_neg/A_partial'])]['stay_id'].unique())

time: 78 ms (started: 2023-07-26 19:19:26 +08:00)


In [276]:
model_pts=list(final_pt_df2[final_pt_df2['final_bin'].isin(['C_pos/A_full','C_neg/A_partial'])]['stay_id'].unique())
dfs[pt_info_df][(dfs[pt_info_df]['label']=='gender')& (dfs[pt_info_df]['stay_id'].isin(model_pts))]['value'].value_counts()
# value
# M    9019
# F    7099

value
M    9019
F    7099
Name: count, dtype: int64

time: 15 ms (started: 2023-07-26 19:20:59 +08:00)


In [277]:
len(model_pts)#16118

16118

time: 0 ns (started: 2023-07-26 19:21:27 +08:00)


In [138]:
#### gender qc

time: 868 µs


In [278]:
##adjusting ages over 90 (which were set to 300 to deidentify) to 90
admit_index=dfs[pt_info_df].loc[(dfs[pt_info_df]['label']=='yearsold')].index
age_tf=pd.to_numeric(dfs[pt_info_df].loc[admit_index,'value'])>90
dfs[pt_info_df].loc[(dfs[pt_info_df]['label']=='yearsold')&(age_tf),'value']=90.0

time: 47 ms (started: 2023-07-26 19:21:34 +08:00)


In [279]:
dfs[pt_info_df].loc[admit_index,'value'].value_counts()

value
90.0    968
67      677
66      651
63      641
68      638
       ... 
22       80
21       68
20       67
19       47
18       19
Name: count, Length: 73, dtype: int64

time: 15 ms (started: 2023-07-26 19:21:37 +08:00)


In [141]:
#dfs[pt_info_df].loc[(dfs[pt_info_df]['label']=='first_admit_age') & (dfs[pt_info_df]['value']>90)]

time: 833 µs


In [280]:
#date= '22102018'
save_df(dfs[pt_info_df], 'pt_info')

del(dfs[pt_info_df] )

time: 282 ms (started: 2023-07-26 19:21:53 +08:00)
